<a href="https://colab.research.google.com/github/Vagarh/Master-in-Data-Science-assignments/blob/main/Procesamiento_LNN_tarea1_2025_1_eafit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Estudiante: Juan Felipe Cardona Arango
* Curso :Procesamiento de Lenguaje Natural
* Tarea 1

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


En estos retos, se están tratando Modelos de Lenguaje (Language Models), principalmente n-gramas y su uso para diferentes aplicaciones:

Cálculo de perplejidad y comparación de modelos.
Cálculo de la probabilidad de una frase.
Generación de texto de manera probabilística.
Clasificación de textos en función de su similitud con distintos dominios, usando perplejidad.
Los n-gramas son un método estadístico clásico para modelar el lenguaje. Un n-grama de tamaño
𝑛
n te dice la probabilidad de la siguiente palabra considerando las
𝑛
−
1
n−1 palabras previas. Por ejemplo:

Unigrama (n=1): cada palabra se asume independiente.
Bigramas (n=2): cada palabra depende de la anterior.
Trigramas (n=3): cada palabra depende de las dos anteriores.
Para cada reto, se han dado diferentes objetivos y pasos:

In [2]:
# Preparación del entorno de trabajo: librería KenLM
#
# KenLM es una biblioteca optimizada para el entrenamiento y consulta de modelos de lenguaje basados en n-gramas.
# Cuenta con varios módulos clave:
#
# - `lmplz`: Se utiliza para entrenar modelos de lenguaje a partir de un corpus de texto, generando modelos en formato ARPA,
#   que pueden ser usados para consultas o conversiones.
# - `build_binary`: Convierte un modelo en formato ARPA a un formato binario, optimizando la consulta del modelo.
# - `query`: Permite realizar consultas en un modelo de lenguaje para evaluar la probabilidad de ocurrencia de una secuencia de palabras.
#
# Aplicaciones clave de KenLM:
# - Reconocimiento de voz
# - Corrección ortográfica
# - Traducción automática
# - Generación de texto
# - Desarrollo de chatbots

!git clone https://github.com/kpu/kenlm.git # clonar el repositorio de kenlm desde github
%cd kenlm # cambiar el directorio
!mkdir build # crear un directorio llamado build para complilar el codigo fuente
%cd build # cambiar a directorio build
!cmake ..
# Configurar el entorno de compilación utilizando CMake
# CMake genera los archivos de configuración necesarios para compilar el proyecto
!make -j4
# Compilar KenLM utilizando Make con la opción "-j4"
# La opción "-j4" permite usar hasta 4 núcleos de la CPU para acelerar la compilación
%cd ..
# Regresar al directorio principal de KenLM después de la compilación


Cloning into 'kenlm'...
remote: Enumerating objects: 14170, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 14170 (delta 85), reused 56 (delta 56), pack-reused 14047 (from 4)
Receiving objects: 100% (14170/14170), 5.98 MiB | 18.60 MiB/s, done.
Resolving deltas: 100% (8041/8041), done.
[Errno 2] No such file or directory: 'kenlm # cambiar el directorio'
/content
[Errno 2] No such file or directory: 'build # cambiar a directorio build'
/content
CMake Warning:
  Ignoring extra path from command line:

   ".."


CMake Error: The source directory "/" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No targets specified and no makefile found.  Stop.
/


In [1]:
# =============================
# Instala librerías del sistema y compila KenLM en un solo paso
# =============================

# 1. Instalar dependencias del sistema
!apt-get update -y
!apt-get install -y build-essential libboost-program-options-dev libboost-system-dev libboost-thread-dev libboost-test-dev libboost-filesystem-dev zlib1g-dev

# 2. Clonar el repositorio de KenLM
!git clone https://github.com/kpu/kenlm.git

# 3. Cambiar al directorio kenlm
%cd kenlm

# 4. Crear y cambiar al directorio build
!mkdir build
%cd build

# 5. Ejecutar CMake
!cmake ..

# 6. Compilar con 4 hilos
!make -j 4

# 7. (Opcional) Instalar (requiere privilegios sudo)
# !sudo make install

# 8. Verifica la presencia de los binarios
!ls bin

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,610 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main

In [5]:
# librerias generales para le procesamiento de texto y nltk
import nltk
import re # Regular expression que permite trabajar con expresiones para manipular cadenas de texto ( util para limpieza de datos, extraccion de informacion, validacion de patrones)

In [ ]:
# definicion del corpus de texto



# Reto 1: Construcción de un modelo de lenguaje n-gram con KenLM

Construcción de un modelo de lenguaje n-gram con KenLM
Objetivo: Entrenar un modelo de lenguaje usando tu propio corpus de texto
Tareas:
1. Descarga un corpus de texto (por ejemplo, Wikipedia, libros en dominio público).
2. Preprocesa el texto: normalización, tokenización, eliminación de caracteres especiales.
3. Usa KenLM para entrenar un modelo de lenguaje con diferentes valores de n (ej. trigramas y pentagramas).
4. Evalúa el modelo calculando la perplejidad sobre un conjunto de prueba (INVESTIGARLO)

In [2]:
def preprocess_text(text):
    # 1️⃣ Convertir a minúsculas
    text = text.lower()

    # 2️⃣ Eliminar caracteres no deseados
    # Se usa una expresión regular para eliminar todo lo que no sea:
    # - Letras minúsculas (a-z)
    # - Números (0-9)
    # - Espacios, comas, puntos, signos de exclamación e interrogación
    text = re.sub(r"[^a-z0-9\s,.!?]", "", text)

    # 3️⃣ Reemplazar múltiples espacios por uno solo
    # Si hay varios espacios seguidos, los reduce a uno solo
    text = re.sub(r"\s+", " ", text)

    # 4️⃣ Eliminar espacios en blanco al inicio y al final
    text = text.strip()

    return text


In [6]:

# Leer y preprocesar el archivo
with open("/content/drive/MyDrive/pg84.txt", "r", encoding="utf-8") as f_in, \
     open("corpus.txt", "w", encoding="utf-8") as f_out:
    for line in f_in:
        clean_line = preprocess_text(line)
        if clean_line:  # Si la línea no está vacía
            f_out.write(clean_line + "\n")


In [7]:
!/content/kenlm/build/bin/lmplz -o 3 < corpus.txt > modelo_3.arpa

# Entrenamiento de un modelo de lenguaje basado en n-gramas utilizando KenLM

# Ejecuta el binario `lmplz` desde la ruta donde se compiló KenLM
# - `-o 3`: Especifica el orden del modelo de lenguaje, en este caso, un modelo de **trigramas (3-gramas)**
# - `< corpus.txt`: Usa el archivo `corpus.txt` como fuente de entrenamiento (entrada estándar)
# - `> modelo_3.arpa`: Guarda el modelo entrenado en un archivo en formato ARPA, que es el estándar para modelos de lenguaje basados en n-gramas


=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 78089 types 10653
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:127836 2:3786927616 3:7100489728
Statistics:
1 10653 D1=0.610175 D2=1.11921 D3+=1.62761
2 48434 D1=0.810595 D2=1.21123 D3+=1.51765
3 70320 D1=0.92044 D2=1.32054 D3+=1.51263
Memory estimate for binary LM:
type      kB
probing 2641 assuming -p 1.5
probing 2967 assuming -r models -p 1.5
trie    1191 without quantization
trie     719 assuming -q 8 -b 8 quantization 
trie    1135 assuming -a 22 array pointer compression
trie     663 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:127836 2:774944 3:1406400
----5---10---15---

In [8]:
# === 1/5 Conteo y ordenamiento de n-gramas ===
# - Se lee el archivo de texto `corpus.txt`.
# - Se cuentan las ocurrencias de unigramas, bigramas y trigramas.
# - Se muestra una barra de progreso indicando el avance del proceso.
# - Se imprime el número total de tokens (palabras) y el tamaño del vocabulario (palabras únicas).

# === 2/5 Cálculo y ordenamiento de conteos ajustados ===
# - Se ajustan los conteos de los n-gramas usando la técnica de Good-Turing.
# - Se generan estructuras de datos para manejar unigramas, bigramas y trigramas.
# - Se imprimen parámetros de descuento (D1, D2, D3+) usados en la estimación de probabilidades.
# - Se muestra una estimación de memoria requerida para almacenar el modelo.

# === 3/5 Cálculo y ordenamiento de probabilidades iniciales ===
# - Se calculan probabilidades iniciales para cada n-grama basado en sus conteos ajustados.
# - Se generan estructuras de datos optimizadas para almacenamiento de probabilidades.
# - Se muestra una barra de progreso indicando el avance del cálculo.

# === 4/5 Cálculo de probabilidades interpoladas ===
# - Se aplican técnicas de interpolación (como Kneser-Ney) para mejorar la precisión del modelo.
# - Se optimizan las probabilidades de los n-gramas para mejorar predicciones en textos futuros.
# - Se muestran estructuras de datos utilizadas en esta etapa.

# === 5/5 Escritura del modelo en formato ARPA ===
# - Se guarda el modelo en un archivo `.arpa`, un formato estándar para modelos de lenguaje.
# - Se muestra una barra de progreso indicando la finalización del proceso.
# - Se imprimen estadísticas del uso de memoria y CPU durante la ejecución.


In [9]:
!/content/kenlm/build/bin/lmplz -o 5 < corpus.txt > modelo_5.arpa

# Entrenamiento de un modelo de lenguaje basado en n-gramas utilizando KenLM

# Ejecuta el binario `lmplz` desde la ruta donde se compiló KenLM
# - `-o 3`: Especifica el orden del modelo de lenguaje, en este caso, un modelo de **trigramas (5-gramas)**
# - `< corpus.txt`: Usa el archivo `corpus.txt` como fuente de entrenamiento (entrada estándar)
# - `> modelo_5.arpa`: Guarda el modelo entrenado en un archivo en formato ARPA, que es el estándar para modelos de lenguaje basados en n-gramas


=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 78089 types 10653
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:127836 2:1062186944 3:1991600640 4:3186561024 5:4647068160
Statistics:
1 10653 D1=0.610175 D2=1.11921 D3+=1.62761
2 48434 D1=0.810595 D2=1.21123 D3+=1.51765
3 70320 D1=0.928497 D2=1.35953 D3+=1.61512
4 70133 D1=0.980818 D2=1.61375 D3+=2.11837
5 64454 D1=0.99341 D2=1.80412 D3+=1.86467
Memory estimate for binary LM:
type      kB
probing 5830 assuming -p 1.5
probing 6978 assuming -r models -p 1.5
trie    2762 without quantization
trie    1505 assuming -q 8 -b 8 quantization 
trie    2541 assuming -a 22 array pointer compression
trie    1284 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3

luego de la creacion de los modelos arpa que fueron dos para 3 n-gram y 5 -n gram se procedioa  convertirlos en binarios optimiza su timepo de consulta y consumo de memoria

In [10]:
!/content/kenlm/build/bin/build_binary modelo_3.arpa /content/drive/MyDrive/modelo_3.bin

# Conversión del modelo de lenguaje ARPA a formato binario optimizado

# - Se ejecuta el binario `build_binary` de KenLM.
# - `modelo_3.arpa`: Archivo de entrada en formato ARPA, que contiene el modelo de lenguaje entrenado.
# - `/content/drive/MyDrive/modelo_3.bin`: Archivo de salida en formato binario, almacenado en Google Drive.
# - El formato binario permite consultas más rápidas y eficientes en comparación con el formato ARPA.
# - Se recomienda esta conversión para su uso en aplicaciones de procesamiento de lenguaje natural.



Reading modelo_3.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [11]:
# convertir en binario el modelo 5 gram
!/content/kenlm/build/bin/build_binary modelo_5.arpa /content/drive/MyDrive/modelo_5.bin

# Conversión del modelo de lenguaje ARPA a formato binario optimizado

# - Se ejecuta el binario `build_binary` de KenLM.
# - `modelo_5.arpa`: Archivo de entrada en formato ARPA, que contiene el modelo de lenguaje entrenado.
# - `/content/drive/MyDrive/modelo_5.bin`: Archivo de salida en formato binario, almacenado en Google Drive.
# - El formato binario permite consultas más rápidas y eficientes en comparación con el formato ARPA.
# - Se recomienda esta conversión para su uso en aplicaciones de procesamiento de lenguaje natural.



Reading modelo_5.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [12]:

!/content/kenlm/build/bin/query /content/drive/MyDrive/modelo_5.bin < /content/drive/MyDrive/pg84.txt

# Evaluación de la perplejidad del modelo de lenguaje (modelo de 5-gramas)

# - Se utiliza el binario `query` de KenLM para evaluar el modelo entrenado.
# - `/content/drive/MyDrive/modelo_5.bin`: Modelo de lenguaje en formato binario.
# - `< /content/drive/MyDrive/pg84.txt`: Archivo de texto de prueba (entrada), sobre el cual se calculará la perplejidad.
# - La perplejidad (PPL) mide qué tan bien el modelo predice el texto de prueba:
#   - PPL alta (1683.42) → El modelo no generaliza bien y tiene dificultades para predecir secuencias.
# - Se analiza el porcentaje de palabras fuera del vocabulario (OOV - Out of Vocabulary):
#   - OOV = 193 palabras (22.6%) → Indica que el corpus de entrenamiento no cubrió bien el vocabulario del texto de prueba.
# - Para mejorar el modelo, se recomienda:
#   - Entrenar con un corpus más grande y representativo.
#   - Aumentar la cobertura del vocabulario durante el entrenamiento.
#   - Considerar modelos de menor orden si hay escasez de datos.

Se han truncado las últimas 5000 líneas del flujo de salida.
retired=1796 2 -3.537696	for=14 3 -1.3626292	the=3 4 -0.638837	night,=429 5 -1.5634956	I=0 1 -4.9238596	took=2090 1 -3.4216032	the=3 2 -1.4298736	boat=5034 3 -1.606779	and=21 4 -0.7259871	passed=493 5 -1.2888184	many=355 5 -1.1287603	hours=1347 5 -0.9065659	upon=180 5 -1.3515191	the=3 5 -0.57431567	</s>=2 5 -0.8579463	Total: -25.318686 OOV: 1
water.=5771 2 -3.8771436	Sometimes,=0 1 -4.820073	with=29 1 -2.0636146	my=155 2 -1.4754609	sails=5772 3 -2.4910884	set,=4037 4 -0.5819682	I=0 1 -4.831356	was=419 1 -2.2118878	carried=5773 2 -2.9761283	by=92 3 -1.331464	the=3 4 -0.49220622	wind;=0 1 -5.1018057	and=21 1 -1.4127642	</s>=2 2 -1.0083745	Total: -34.675335 OOV: 3
sometimes,=3806 2 -4.090964	after=1112 3 -1.0265915	rowing=4445 4 -1.0352371	into=487 5 -0.73929083	the=3 5 -0.4079666	middle=5774 5 -1.5646176	of=7 5 -0.3310933	the=3 5 -0.35839114	lake,=2529 5 -1.7233686	I=0 1 -4.831356	left=2848 1 -3.5936155	the=3 2 -1.6614819	boat=

In [13]:
!/content/kenlm/build/bin/query /content/drive/MyDrive/modelo_3.bin < /content/drive/MyDrive/pg84.txt
# Evaluación de la perplejidad del modelo de 3-gramas con un conjunto de prueba

# - Se usa el binario `query` de KenLM para evaluar el desempeño del modelo de lenguaje.
# - `/content/drive/MyDrive/modelo_3.bin`: Modelo de lenguaje en formato binario (3-gramas).
# - `< /content/drive/MyDrive/pg84.txt`: Archivo de texto de prueba utilizado para calcular la perplejidad.
# - La perplejidad (PPL) mide la capacidad del modelo para predecir secuencias de palabras:
#   - PPL = 1685.49 (muy alta) → Indica que el modelo tiene dificultades para generalizar.
# - Se analiza la cantidad de palabras fuera del vocabulario (OOV - Out of Vocabulary):
#   - OOV = 193 palabras (22.6%) → Indica que una gran cantidad de palabras en el texto de prueba no estaban en el corpus de entrenamiento.
# - Recomendaciones para mejorar el modelo:
#   - Aumentar el tamaño y diversidad del corpus de entrenamiento.
#   - Considerar el uso de un modelo de mayor orden (5-gramas) si hay suficientes datos.
#   - Aplicar técnicas de suavizado para mejorar la estimación de probabilidades en n-gramas raros.

Se han truncado las últimas 5000 líneas del flujo de salida.
retired=1796 2 -3.537696	for=14 3 -1.338149	the=3 3 -0.6547219	night,=429 3 -2.747292	I=0 1 -4.916362	took=2090 1 -3.4216032	the=3 2 -1.4298736	boat=5034 3 -1.5621281	and=21 3 -0.7467011	passed=493 3 -1.3951043	many=355 3 -1.2685356	hours=1347 3 -0.9812579	upon=180 3 -1.686086	the=3 3 -0.5956478	</s>=2 3 -0.92468125	Total: -27.20584 OOV: 1
water.=5771 2 -3.8771436	Sometimes,=0 1 -4.8238583	with=29 1 -2.0636146	my=155 2 -1.4754609	sails=5772 3 -2.4465537	set,=4037 3 -0.5952829	I=0 1 -4.8238583	was=419 1 -2.2118878	carried=5773 2 -2.9761283	by=92 3 -1.3088828	the=3 3 -0.5021142	wind;=0 1 -5.095837	and=21 1 -1.4127642	</s>=2 2 -1.0083745	Total: -34.62176 OOV: 3
sometimes,=3806 2 -4.090964	after=1112 3 -0.9917327	rowing=4445 3 -1.0836284	into=487 3 -0.77462053	the=3 3 -0.41991553	middle=5774 3 -2.7478142	of=7 3 -0.33982766	the=3 3 -0.3566607	lake,=2529 3 -2.315402	I=0 1 -4.8238583	left=2848 1 -3.5936155	the=3 2 -1.6614819	boat=50

Observaciones
ambos modelos tienen un desempeño muy similar, y en general, no están funcionando bien. ¿Por qué?
* Tienen una perplejidad muy alta (>1000), lo que significa que les cuesta mucho predecir el texto correctamente.
*  Ambos fallan con las mismas palabras desconocidas (OOVs), lo que indica que no han aprendido suficiente vocabulario.

🧐 ¿Qué está pasando?
el modelo fue entrenado con un conjunto de datos muy limitado.Es como si alguien aprendiera español solo con frases exactas y luego intentara entender un texto completamente nuevo… No lo reconoce y se confunde.

* ¿Cómo mejorar esto?
1️⃣ Usa más datos de entrenamiento para que el modelo aprenda más palabras.
2️⃣ Prueba con un modelo de menor orden (como 3-gram en lugar de 5-gram) para que pueda generalizar mejor.
3️⃣ Aplica suavizado (como Kneser-Ney) para manejar mejor las palabras desconocidas.

🔹 En resumen: El modelo no ha visto suficientes ejemplos y se pierde con textos nuevos. Necesita más datos y un ajuste en la técnica para mejorar.


# Reto 2: Evaluación de Modelos de Lenguaje

Evaluación de Modelos de Lenguaje (INVESTIGACIÓN)
Objetivo: Comparar la calidad de diferentes modelos de lenguaje.
Tareas:
1. Entrena modelos con diferentes valores de n (ej. unigramas, bigramas, trigramas).
2. Calcula la perplejidad sobre un conjunto de validación.
3. Compara la perplejidad y analiza cómo cambia con el tamaño del n-grama

In [14]:
# Entrenamiento de un modelo de lenguaje basado en bigramas (n=2)

# - Se ejecuta el binario `lmplz` de KenLM para entrenar el modelo.
# - `-o 2`: Especifica que se generará un modelo de **bigramas** (cada predicción depende de la palabra anterior).
# - `< corpus.txt`: Usa el archivo `corpus.txt` como fuente de entrenamiento (entrada estándar).
# - `> modelo_2.arpa`: Guarda el modelo entrenado en un archivo en formato ARPA, que es el estándar para modelos de lenguaje basados en n-gramas.

# 🔹 Un modelo de bigramas puede generalizar mejor que uno de trigramas o 5-gramas cuando hay pocos datos,
#    ya que reduce la cantidad de combinaciones posibles y disminuye la cantidad de palabras desconocidas (OOVs).

!/content/kenlm/build/bin/lmplz -o 2 < corpus.txt > modelo_2.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 78089 types 10653
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:127836 2:10887416832
Statistics:
1 10653 D1=0.610175 D2=1.11921 D3+=1.62761
2 48434 D1=0.797774 D2=1.17145 D3+=1.44169
Memory estimate for binary LM:
type      kB
probing 1121 assuming -p 1.5
probing 1163 assuming -r models -p 1.5
trie     515 without quantization
trie     380 assuming -q 8 -b 8 quantization 
trie     515 assuming -a 22 array pointer compression
trie     380 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:127836 2:774944
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80--

In [15]:
# Entrenamiento de un modelo de lenguaje basado en trigramas (n=3)

# - Se ejecuta el binario `lmplz` de KenLM para entrenar el modelo.
# - `-o 3`: Especifica que se generará un modelo de **trigramas** (cada predicción depende de las dos palabras anteriores).
# - `< corpus.txt`: Usa el archivo `corpus.txt` como fuente de entrenamiento (entrada estándar).
# - `> modelo_3.arpa`: Guarda el modelo entrenado en un archivo en formato ARPA, que es el estándar para modelos de lenguaje basados en n-gramas.

# 🔹 Un modelo de trigramas tiene mayor precisión que un modelo de bigramas, pero requiere más datos para evitar problemas
#    con palabras fuera del vocabulario (OOVs) y mejorar la generalización.
# 🔹 Si el corpus de entrenamiento es pequeño, la perplejidad del modelo podría ser alta debido a la falta de ejemplos.
!/content/kenlm/build/bin/lmplz -o 3 < corpus.txt > modelo_3.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 78089 types 10653
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:127836 2:3786927616 3:7100489728
Statistics:
1 10653 D1=0.610175 D2=1.11921 D3+=1.62761
2 48434 D1=0.810595 D2=1.21123 D3+=1.51765
3 70320 D1=0.92044 D2=1.32054 D3+=1.51263
Memory estimate for binary LM:
type      kB
probing 2641 assuming -p 1.5
probing 2967 assuming -r models -p 1.5
trie    1191 without quantization
trie     719 assuming -q 8 -b 8 quantization 
trie    1135 assuming -a 22 array pointer compression
trie     663 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:127836 2:774944 3:1406400
----5---10---15---

In [16]:
# Entrenamiento de un modelo de lenguaje basado en pentagramas (n=5)

# - Se ejecuta el binario `lmplz` de KenLM para entrenar el modelo.
# - `-o 5`: Especifica que se generará un modelo de **pentagramas** (cada predicción depende de las cuatro palabras anteriores).
# - `< corpus.txt`: Usa el archivo `corpus.txt` como fuente de entrenamiento (entrada estándar).
# - `> modelo_5.arpa`: Guarda el modelo entrenado en un archivo en formato ARPA, que es el estándar para modelos de lenguaje basados en n-gramas.

# 🔹 Un modelo de pentagramas puede capturar mejor la estructura gramatical y el contexto de las frases,
#    pero tiene un costo computacional mayor y requiere un corpus extenso para evitar problemas de perplejidad alta.
# 🔹 Si el conjunto de entrenamiento es limitado, el modelo puede no generalizar bien y tener un alto número de palabras fuera del vocabulario (OOVs).
# 🔹 Se recomienda aplicar técnicas de suavizado (como Kneser-Ney) para mejorar el rendimiento del modelo.
!/content/kenlm/build/bin/lmplz -o 5 < corpus.txt > modelo_5.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 78089 types 10653
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:127836 2:1062186944 3:1991600640 4:3186561024 5:4647068160
Statistics:
1 10653 D1=0.610175 D2=1.11921 D3+=1.62761
2 48434 D1=0.810595 D2=1.21123 D3+=1.51765
3 70320 D1=0.928497 D2=1.35953 D3+=1.61512
4 70133 D1=0.980818 D2=1.61375 D3+=2.11837
5 64454 D1=0.99341 D2=1.80412 D3+=1.86467
Memory estimate for binary LM:
type      kB
probing 5830 assuming -p 1.5
probing 6978 assuming -r models -p 1.5
trie    2762 without quantization
trie    1505 assuming -q 8 -b 8 quantization 
trie    2541 assuming -a 22 array pointer compression
trie    1284 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3

 📌 Comparación de Modelos de Lenguaje: Bigramas, Trigramas y Pentagramas

| Modelo      | Pros ✅ | Contras ❌ |
|------------|--------|-----------|
| **Bigramas (n=2)** | ✅ **Eficiente en memoria y cómputo**: Requiere menos espacio y es más rápido que trigramas y pentagramas. <br> ✅ **Mejor generalización con pocos datos**: Puede manejar vocabularios más pequeños sin sobreajustarse. <br> ✅ **Menos problemas de palabras fuera del vocabulario (OOVs)**: Menos combinaciones posibles. | ❌ **Falta de contexto**: Solo considera la palabra anterior, lo que puede generar errores en la predicción. <br> ❌ **Menor precisión**: No captura bien estructuras gramaticales complejas ni relaciones a largo plazo. |
| **Trigramas (n=3)** | ✅ **Mejor comprensión del contexto**: Considera dos palabras anteriores, lo que mejora la predicción. <br> ✅ **Mayor precisión que bigramas**: Se adapta mejor a frases y patrones lingüísticos. <br> ✅ **Buen equilibrio entre calidad y eficiencia**: Si el corpus es lo suficientemente grande, ofrece un rendimiento decente sin un costo computacional extremo. | ❌ **Mayor consumo de memoria y cómputo**: Requiere más almacenamiento y procesamiento que los bigramas. <br> ❌ **Puede sufrir de escasez de datos**: Si el corpus es pequeño, muchos trigramas no estarán en el entrenamiento, aumentando la perplejidad. |
| **Pentagramas (n=5)** | ✅ **Máxima captura de contexto**: Analiza hasta cuatro palabras previas, lo que permite predicciones más precisas y con mejor estructura gramatical. <br> ✅ **Útil en corpus grandes**: En contextos con datos masivos, puede mejorar la fluidez de los modelos de lenguaje. <br> ✅ **Beneficioso en tareas como generación de texto y corrección gramatical**. | ❌ **Altos requerimientos computacionales**: Usa mucha memoria y procesamiento, lo que puede ser problemático en dispositivos con recursos limitados. <br> ❌ **Más palabras fuera del vocabulario (OOVs)**: Si el corpus es pequeño, habrá muchas combinaciones no vistas, aumentando la perplejidad. <br> ❌ **Sobreajuste**: Puede memorizar patrones en lugar de generalizar, haciendo que el modelo sea menos flexible con textos nuevos. |

---

 🔹 **¿Cuál modelo elegir?**

1️⃣ **Bigramas (n=2)** → Si el corpus es pequeño y se necesita eficiencia. Ideal para casos con recursos limitados.  
2️⃣ **Trigramas (n=3)** → Buen equilibrio entre precisión y costo computacional. Útil para tareas como reconocimiento de voz o autocompletado.  
3️⃣ **Pentagramas (n=5)** → Si hay un corpus muy grande y se necesita máxima precisión, como en generación de texto avanzada.  



In [17]:
# Conversión de modelos de lenguaje ARPA a formato binario optimizado

# - Se ejecuta el binario `build_binary` de KenLM para convertir los modelos en formato ARPA a formato binario.
# - `modelo_X.arpa`: Archivo de entrada en formato ARPA (X indica el orden del modelo: 2, 3 o 5).
# - `/content/drive/MyDrive/modelo_X.bin`: Archivo de salida en formato binario, almacenado en Google Drive.

# 🔹 Beneficios de la conversión a binario:
#   - **Mayor velocidad de consulta**: El formato binario permite búsquedas más rápidas que el formato ARPA.
#   - **Menor consumo de memoria**: Se optimiza el almacenamiento del modelo, haciéndolo más eficiente en tiempo de ejecución.
#   - **Recomendado para aplicaciones en producción**: Si el modelo será utilizado en tareas de NLP como reconocimiento de voz o corrección gramatical,
#     el formato binario es más adecuado.

# 🔹 Uso posterior:
#   - Estos modelos binarios pueden ser evaluados con el comando `query` de KenLM para calcular perplejidad y analizar su desempeño.

# Convertir los modelos de bigramas, trigramas y pentagramas a formato binario

!/content/kenlm/build/bin/build_binary modelo_2.arpa /content/drive/MyDrive/modelo_2.bin
!/content/kenlm/build/bin/build_binary modelo_3.arpa /content/drive/MyDrive/modelo_3.bin
!/content/kenlm/build/bin/build_binary modelo_5.arpa /content/drive/MyDrive/modelo_5.bin


Reading modelo_2.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS
Reading modelo_3.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS
Reading modelo_5.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [18]:
# Evaluación de modelos de lenguaje y almacenamiento de resultados en archivos de texto

# - Se ejecuta el binario `query` de KenLM para evaluar cada modelo de lenguaje.
# - `/content/drive/MyDrive/modelo_X.bin`: Modelo de lenguaje en formato binario (X indica el orden del modelo: 2, 3 o 5).
# - `< /content/drive/MyDrive/Analitica/texto_prueba.txt`: Archivo de prueba utilizado para calcular la perplejidad y analizar el desempeño del modelo.
# - `> resultado_modelo_X.txt`: Guarda los resultados de la evaluación en un archivo de texto para su posterior análisis.

# 🔹 El resultado incluye:
#   - Perplejidad (PPL) → Indica qué tan bien el modelo predice las secuencias del texto de prueba.
#   - Cantidad de palabras fuera del vocabulario (OOVs) → Mide cuántas palabras en el texto de prueba no fueron vistas en el entrenamiento.
#   - Ocurrencias de n-gramas conocidos y desconocidos.

# 🔹 ¿Cómo analizar los resultados?
#   - Comparar la perplejidad entre los modelos (un valor menor indica mejor capacidad de predicción).
#   - Evaluar el porcentaje de palabras fuera del vocabulario (OOVs) para detectar deficiencias en el corpus de entrenamiento.
#   - Determinar si modelos más complejos (pentagramas) realmente mejoran el rendimiento o si un modelo más simple (bigramas/trigramas) es suficiente.

# Ejecutar evaluación y guardar salida en archivos de texto
!/content/kenlm/build/bin/query /content/drive/MyDrive/modelo_2.bin < /content/drive/MyDrive/Analitica/texto_prueba.txt > resultado_modelo_2.txt
!/content/kenlm/build/bin/query /content/drive/MyDrive/modelo_3.bin < /content/drive/MyDrive/Analitica/texto_prueba.txt > resultado_modelo_3.txt
!/content/kenlm/build/bin/query /content/drive/MyDrive/modelo_5.bin < /content/drive/MyDrive/Analitica/texto_prueba.txt > resultado_modelo_5.txt



This binary file contains probing hash tables.
Name:query	VmPeak:8552 kB	VmRSS:4580 kB	RSSMax:5704 kB	user:0.019132	sys:0.002869	CPU:0.0220561	real:0.281379
This binary file contains probing hash tables.
Name:query	VmPeak:10072 kB	VmRSS:4524 kB	RSSMax:7168 kB	user:0.015097	sys:0.009058	CPU:0.0241989	real:0.0285352
This binary file contains probing hash tables.
Name:query	VmPeak:13260 kB	VmRSS:4432 kB	RSSMax:10264 kB	user:0.024918	sys:0.011075	CPU:0.0360514	real:0.0452322


In [19]:
import re

# Función para extraer el valor de perplejidad
def extraer_perplejidad(filename):
    with open(filename, 'r') as f:
        texto = f.read()

    match = re.search(r"Perplexity including OOVs:\s+([\d\.]+)", texto)
    if match:
        return float(match.group(1))
    return None

# Extraer perplejidad de cada modelo
perplejidad_modelo_2 = extraer_perplejidad("resultado_modelo_2.txt")
perplejidad_modelo_3 = extraer_perplejidad("resultado_modelo_3.txt")
perplejidad_modelo_5 = extraer_perplejidad("resultado_modelo_5.txt")

# Mostrar resultados
print(f"Perplejidad del modelo 2-gramas: {perplejidad_modelo_2}")
print(f"Perplejidad del modelo 3-gramas: {perplejidad_modelo_3}")
print(f"Perplejidad del modelo 5-gramas: {perplejidad_modelo_5}")


Perplejidad del modelo 2-gramas: 210.05581796500664
Perplejidad del modelo 3-gramas: 81.29083283563767
Perplejidad del modelo 5-gramas: 65.61473015159368


 📊 **Análisis de Perplejidad de los Modelos de Lenguaje**

| Modelo      | Perplejidad 📉 |
|------------|--------------|
| **2-gramas** | 210.06 |
| **3-gramas** | 81.29 |
| **5-gramas** | 65.61 |

---

 🔍 **Interpretación de los resultados**

 🟠 **Bigramas (n=2) → Perplejidad: 210.06**

- Es el modelo con **mayor perplejidad**, lo que indica que tiene **dificultades para predecir correctamente** el texto de prueba.
- Considera solo una palabra previa, lo que **limita su capacidad de capturar contexto**.
- Puede ser útil si se dispone de pocos datos, pero en este caso no generaliza bien.

 🟡 **Trigramas (n=3) → Perplejidad: 81.29**
- **Mejora significativa** en comparación con el modelo de bigramas.
- Al considerar dos palabras anteriores, **capta mejor el contexto** de las oraciones.
- Es un buen equilibrio entre precisión y costo computacional.


 🟢 **Pentagramas (n=5) → Perplejidad: 65.61**
- Tiene la **perplejidad más baja**, lo que indica **mejor predicción del texto** de prueba.
- Sin embargo, este modelo requiere **más memoria y procesamiento** y puede sobreajustarse si el corpus de entrenamiento es pequeño.

---

 🧐 **Conclusión y recomendaciones**
✅ **El modelo de pentagramas (5-gramas) tiene el mejor desempeño**, pero es el más costoso en términos computacionales.  
✅ **El modelo de trigramas (3-gramas) ofrece un buen equilibrio** entre precisión y eficiencia.  
✅ **El modelo de bigramas (2-gramas) tiene una perplejidad alta**, lo que sugiere que **no captura suficiente contexto**.


🔹 **¿Cómo mejorar estos modelos?**
1️⃣ Si el corpus de entrenamiento es pequeño, **trigramas (n=3) es la mejor opción**.  
2️⃣ Si se dispone de más datos, **5-gramas es óptimo** para mejorar la precisión.  
3️⃣ **Aplicar suavizado** (como Kneser-Ney) para mejorar la estimación de probabilidades.  
4️⃣ **Ampliar el corpus** para reducir el número de palabras fuera del vocabulario (OOVs).  

🚀 **En resumen:**  
El modelo de **5-gramas predice mejor el texto**, pero si buscas **eficiencia y generalización**, el **modelo de 3-gramas es una excelente alternativa**.


# Reto 3: Probabilidad de una Secuencia de Palabras

Objetivo: Calcular la probabilidad de una oración dada usando un modelo de lenguaje.

Tareas:

1. Entrenar un modelo de 3-gramas.

2. Usar KenLM para obtener la probabilidad de diferentes frases.

3. Comparar las probabilidades entre frases gramaticalmente correctas y frases aleatorias

In [20]:
# Instalación de KenLM desde el repositorio oficial de GitHub

# - Se usa `pip install` para descargar e instalar KenLM directamente desde su repositorio en GitHub.
# - La URL `https://github.com/kpu/kenlm/archive/master.zip` contiene la versión más reciente del código fuente de KenLM.
# - Esto permite instalar KenLM sin necesidad de compilarlo manualmente desde el código fuente.

!pip install https://github.com/kpu/kenlm/archive/master.zip


     | 553.6 kB 3.9 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp311-cp311-linux_x86_64.whl size=3186570 sha256=046d50841800f006b81ff0616dab6bc4888405f460432c36d6afc67cc20cda90
  Stored in directory: /tmp/pip-ephem-wheel-cache-w678so6m/wheels/4e/ca/6a/e5da175b1396483f6f410cdb4cfe8bc8fa5e12088e91d60413
Successfully built kenlm


In [21]:
import kenlm
print(dir(kenlm))


['ARPALoadComplain', 'Config', 'FullScoreReturn', 'LanguageModel', 'LoadMethod', 'Model', 'State', '__builtins__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__test__', 'os']


In [22]:
import kenlm

# Cargar el modelo de lenguaje 3-gramas previamente entrenado
modelo = kenlm.LanguageModel('/content/drive/MyDrive/modelo_3.bin')

# Función para calcular la log-probabilidad y perplejidad de una oración
def evaluate_sentence(sentence):
    log_prob = modelo.score(sentence)  # Log-probabilidad de la frase
    perplexity = 10 ** (-log_prob / len(sentence.split()))  # Perplejidad
    return log_prob, perplexity

# Lista de frases para evaluar
sentences = [
    "El clima está muy agradable hoy.",  # inCorrecta
    "Hoy clima está muy agradable el.",  # Incorrecta
    "El coche rojo es más rápido que el azul.",  # inCorrecta
    "Coche más azul el rápido que rojo es.", # incorrecta
    "There, Margaret, the sun is for evervisible, its broad disk just skirting the horizon and diffusing a perpetual splendour# Incorrecta",# correcta
    "These visions faded when I perused, for the first time, those poets whose effusions entranced my soul and lifted it to heaven.",# correcta
    "The passionate and almost reverential attachment with which all regarded her became, while I shared it, my pride and my delight"

]

# el corpus de texto fue entrado en ingles usando el libreo Title: Frankenstein; Or, The Modern Prometheus

# Evaluar cada frase
for sentence in sentences:
    log_prob, perplexity = evaluate_sentence(sentence)
    print(f"Frase: '{sentence}'")
    print(f"  Log-probabilidad: {log_prob}")
    print(f"  Perplejidad: {perplexity}\n")


Frase: 'El clima está muy agradable hoy.'
  Log-probabilidad: -29.871633529663086
  Perplejidad: 95193.12592134708

Frase: 'Hoy clima está muy agradable el.'
  Log-probabilidad: -29.871633529663086
  Perplejidad: 95193.12592134708

Frase: 'El coche rojo es más rápido que el azul.'
  Log-probabilidad: -43.96160888671875
  Perplejidad: 76669.60214861811

Frase: 'Coche más azul el rápido que rojo es.'
  Log-probabilidad: -39.264949798583984
  Perplejidad: 80931.71152028543

Frase: 'There, Margaret, the sun is for evervisible, its broad disk just skirting the horizon and diffusing a perpetual splendour# Incorrecta'
  Log-probabilidad: -48.802188873291016
  Perplejidad: 275.4922865661442

Frase: 'These visions faded when I perused, for the first time, those poets whose effusions entranced my soul and lifted it to heaven.'
  Log-probabilidad: -43.009002685546875
  Perplejidad: 90.1477034825714

Frase: 'The passionate and almost reverential attachment with which all regarded her became, while

 📊 **Análisis de Log-Probabilidad y Perplejidad en Frases de un Modelo de Lenguaje**

| **Frase** | **Log-Probabilidad 📉** | **Perplejidad 📊** |
|-----------|------------------------|--------------------|
| *El clima está muy agradable hoy.* | -29.87 | 95193.13 |
| *Hoy clima está muy agradable el.* | -29.87 | 95193.13 |
| *El coche rojo es más rápido que el azul.* | -43.96 | 76669.60 |
| *Coche más azul el rápido que rojo es.* | -39.26 | 80931.71 |
| *There, Margaret, the sun is for evervisible...* (**Incorrecta**) | -48.80 | 275.49 |
| *These visions faded when I perused...* | -43.00 | 90.15 |
| *The passionate and almost reverential attachment...* | -49.69 | 232.55 |

---

 🔍 **Interpretación de los resultados**

 🛠 **1. Análisis de Log-Probabilidad**
- La **log-probabilidad** mide qué tan probable es una frase según el modelo.
- Valores más **negativos** indican **menor probabilidad** y, por lo tanto, peor ajuste al modelo.
- Frases estructuradas correctamente en español tienen **log-probabilidades similares**, mientras que en inglés varían más dependiendo del contenido.

 📈 **2. Análisis de Perplejidad**
- La **perplejidad** (PPL) mide qué tan bien el modelo predice la secuencia de palabras:
  - **Valores bajos** → El modelo predice bien la secuencia de palabras.
  - **Valores altos** → El modelo tiene dificultades para predecir la secuencia, lo que indica que **no ha sido bien entrenado en ese tipo de texto**.

 🟠 **3. Comparación entre frases en español**
- Las frases *"El clima está muy agradable hoy."* y *"Hoy clima está muy agradable el."* tienen **exactamente la misma perplejidad**, lo que indica que el modelo **no ha aprendido correctamente el orden natural del español**.
- La frase *"Coche más azul el rápido que rojo es."* tiene **mejor log-probabilidad** que *"El coche rojo es más rápido que el azul."*, lo que sugiere que el modelo **no ha sido bien entrenado en estructuras gramaticales en español**.

 🔵 **4. Comparación entre frases en inglés**
- La frase **incorrecta** (*"There, Margaret, the sun is for evervisible..."*) tiene una perplejidad **275.49**, indicando que el modelo reconoce errores en la secuencia de palabras.
- La frase con menor perplejidad es **"These visions faded when I perused..."** con **90.15**, lo que sugiere que el modelo ha sido entrenado en textos similares en inglés.

---
 🧐 **Conclusiones y Recomendaciones**
✅ **El modelo de lenguaje tiene deficiencias en español.**  
✅ **El modelo parece estar mejor entrenado en inglés que en español.**  
✅ **Perplejidades extremadamente altas (>90,000) sugieren que el modelo necesita un mejor corpus de entrenamiento en español.**  
✅ **El modelo no está reconociendo correctamente el orden natural de las palabras en español.**  

 🔹 **¿Cómo mejorar el modelo?**
1️⃣ **Ampliar el corpus de entrenamiento en español.**  
2️⃣ **Aplicar técnicas de suavizado (como Kneser-Ney) para mejorar la predicción en español.**  
3️⃣ **Evaluar si el modelo ha sido entrenado con suficientes ejemplos en español y ajustar la cantidad de datos en cada idioma.**  
4️⃣ **Realizar más pruebas con frases estructuradas para medir cómo el modelo responde a diferentes cambios gramaticales.**  

🚀 **En resumen:**  
El modelo está funcionando mejor en inglés que en español, y tiene dificultades con el orden gramatical en frases en español. Se recomienda **expandir el corpus de entrenamiento en español** y **aplicar técnicas de suavizado** para mejorar la generalización. 🎯


# Reto 4. Generación de Texto Basado en Probabilidades

Objetivo: Usar un modelo de lenguaje para generar texto de manera probabilística.

Tareas:

1. Implementar una función que, dado un modelo entrenado, genere texto palabra por palabra.

2. Comparar la coherencia del texto generado con diferentes valores de n.

In [67]:
import kenlm
import re
import math
import random

# ============================================
# 1) CONFIGURACIÓN DE RUTAS Y LECTURA DEL CORPUS
# ============================================
bigram_model_path = '/content/drive/MyDrive/modelo_2.bin'
trigram_model_path = '/content/drive/MyDrive/modelo_3.bin'
pentagram_model_path = '/content/drive/MyDrive/modelo_5.bin'
corpus_path = '/content/drive/MyDrive/pg84.txt'

# ============================================
# 2) CREACIÓN DEL VOCABULARIO A PARTIR DEL CORPUS
# ============================================
vocab_set = set()
with open(corpus_path, 'r', encoding='utf-8') as f:
    for line in f:
        tokens = re.findall(r'\w+', line.lower())
        for token in tokens:
            vocab_set.add(token)

vocabulario = sorted(list(vocab_set))
print(f"Vocabulario creado. Total de palabras únicas: {len(vocabulario)}")

# ============================================
# 3) CARGA DE LOS MODELOS KENLM
# ============================================
bigram_model = kenlm.LanguageModel(bigram_model_path)
trigram_model = kenlm.LanguageModel(trigram_model_path)
pentagram_model = kenlm.LanguageModel(pentagram_model_path)

# ============================================
# 4) FUNCIÓN DE GENERACIÓN CON TOP-k SAMPLING
# ============================================
def generar_texto_sampling(model, inicio, longitud=10, k=5):
    """
    Genera texto usando un modelo KenLM con Top-k sampling.
    :param model: Modelo KenLM cargado.
    :param inicio: Lista de palabras de inicio (semilla).
    :param longitud: Número total de palabras a generar.
    :param k: Número de palabras a considerar en el muestreo (Top-k).
    :return: Lista de palabras generadas.
    """
    texto = inicio[:]

    while len(texto) < longitud:
        order = model.order
        # Contexto: si no alcanza, se toma lo que haya
        if len(texto) < order - 1:
            contexto = texto
        else:
            contexto = texto[-(order - 1):]

        contexto_str = " ".join(contexto)

        # Paso 1: Calcular score de todas las palabras del vocabulario
        # y almacenarlas en una lista (palabra, score).
        scores_candidatos = []
        for palabra in vocabulario:
            candidato_str = contexto_str + " " + palabra
            score = model.score(candidato_str, bos=False, eos=False)
            scores_candidatos.append((palabra, score))

        # Paso 2: Ordenar por score de mayor a menor
        scores_candidatos.sort(key=lambda x: x[1], reverse=True)

        # Paso 3: Seleccionar top-k
        top_k_candidatos = scores_candidatos[:k]

        # Paso 4: Convertir los log-scores a probabilidades "normales"
        # Recuerda que KenLM da log base 10 (en la mayoría de configuraciones).
        # Asegúrate de confirmar la base del log (podría ser base 10 o e).
        # Por convención, la versión actual de KenLM usa log base e (natural).
        # Ver: https://kheafield.com/code/kenlm/estimation/
        # Para estar seguros, asumamos log base e.
        # p = exp(score_log)

        # Primero convertimos log-scores a pesos en la escala normal
        max_score = top_k_candidatos[0][1]  # el mayor log-score del top-k
        exp_probs = []
        for palabra, s in top_k_candidatos:
            # restar max_score para evitar overflow
            exp_prob = math.exp(s - max_score)
            exp_probs.append(exp_prob)

        # Paso 5: Normalizar
        suma = sum(exp_probs)
        probs = [p / suma for p in exp_probs]

        # Paso 6: Muestrear una palabra aleatoriamente según probs
        # random.choices permite selección ponderada en Python 3.6+
        palabras_topk = [pc[0] for pc in top_k_candidatos]
        siguiente_palabra = random.choices(palabras_topk, weights=probs, k=1)[0]

        # Agregar la siguiente palabra
        texto.append(siguiente_palabra)

    return texto

# ============================================
# 5) GENERACIÓN DE TEXTO CON CADA MODELO (Top-k sampling)
# ============================================
semilla = ["robaidlaid"]

print("=== Texto generado con Bigram (Top-k) ===")
texto_bigram = generar_texto_sampling(bigram_model, semilla, longitud=15, k=5)
print(" ".join(texto_bigram))

print("\n=== Texto generado con Trigram (Top-k) ===")
texto_trigram = generar_texto_sampling(trigram_model, semilla, longitud=15, k=5)
print(" ".join(texto_trigram))

print("\n=== Texto generado con Pentagram (Top-k) ===")
texto_pentagram = generar_texto_sampling(pentagram_model, semilla, longitud=15, k=5)
print(" ".join(texto_pentagram))


Vocabulario creado. Total de palabras únicas: 7335
=== Texto generado con Bigram (Top-k) ===
robaidlaid of this and the and the same time to my heart i have been

=== Texto generado con Trigram (Top-k) ===
robaidlaid of a human being in the united and the sun was the same time

=== Texto generado con Pentagram (Top-k) ===
robaidlaid and my eyes of his and to the most and i felt as if


# Reto 5: Detección de Texto Anómalo

Objetivo: Usar modelos n-gram para detectar frases fuera de distribución.

Tareas:

1. Entrenar un modelo en un dominio específico (ej. noticias, literatura).

2. Calcular la probabilidad de frases tomadas de otro dominio (ej. tweets, mensajes informales).

3. Identificar oraciones con baja probabilidad como anomalías.


In [27]:
# entrenamiento de un modelo de tema especifico biologia
# Leer y preprocesar el archivo
with open("/content/drive/MyDrive/Analitica/Dinosauria-Wikipedia_-la-enciclopedia-libre.txt", "r", encoding="utf-8") as f_in, \
     open("corpus_wikipedia.txt", "w", encoding="utf-8") as f_out:
    for line in f_in:
        clean_line = preprocess_text(line)
        if clean_line:  # Si la línea no está vacía
            f_out.write(clean_line + "\n")


In [28]:
!/content/kenlm/build/bin/lmplz -o 3 < corpus_wikipedia.txt > modelo_3_wiki.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/corpus_wikipedia.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 25645 types 8000
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:96000 2:3786938624 3:7100510208
Statistics:
1 8000 D1=0.750528 D2=1.21957 D3+=1.49032
2 19344 D1=0.876312 D2=1.19408 D3+=1.57541
3 22852 D1=0.908448 D2=1.44733 D3+=1.56649
Memory estimate for binary LM:
type      kB
probing 1058 assuming -p 1.5
probing 1202 assuming -r models -p 1.5
trie     525 without quantization
trie     353 assuming -q 8 -b 8 quantization 
trie     506 assuming -a 22 array pointer compression
trie     334 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:96000 2:309504 3:457040
----5---10--

In [29]:
# convertir en binario el modelo 5 gram
!/content/kenlm/build/bin/build_binary modelo_3_wiki.arpa /content/drive/MyDrive/modelo_3_wiki.bin


Reading modelo_3_wiki.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [30]:
# Cargar el modelo de lenguaje entrenado con un corpus de dinosaurios
modelo = kenlm.LanguageModel("/content/drive/MyDrive/modelo_3_wiki.bin")

# Frases del mismo dominio (dinosaurios)
dino_sentences = [
    "El tiranosaurio rex fue uno de los depredadores más temidos.",
    "Los fósiles indican que algunos dinosaurios tenían plumas.",
    "El velociraptor era un cazador ágil y veloz."
]

# Frases de otro dominio (tweets, política, lenguaje informal)
other_sentences = [
    "Hoy el presidente anunció nuevas reformas económicas.",
    "LOL no puedo creer que mi equipo haya perdido de nuevo.",
    "¿Qué opinas sobre la última tendencia en redes sociales?"
]

In [31]:
# Función para calcular la log-probabilidad y perplejidad de una frase
def analyze_sentence(sentence):
    log_prob = modelo.score(sentence)
    perplexity = 10 ** (-log_prob / len(sentence.split()))
    return log_prob, perplexity

In [32]:
# Evaluar frases de dinosaurios
print("\n📌 Probabilidades de frases sobre dinosaurios:")
for sentence in dino_sentences:
    log_prob, perplexity = analyze_sentence(sentence)
    print(f"🦖 '{sentence}' → Log-Prob: {log_prob:.2f}, Perplejidad: {perplexity:.2f}")



📌 Probabilidades de frases sobre dinosaurios:
🦖 'El tiranosaurio rex fue uno de los depredadores más temidos.' → Log-Prob: -33.05, Perplejidad: 2017.77
🦖 'Los fósiles indican que algunos dinosaurios tenían plumas.' → Log-Prob: -28.06, Perplejidad: 3219.02
🦖 'El velociraptor era un cazador ágil y veloz.' → Log-Prob: -30.29, Perplejidad: 6116.94


In [33]:
# Evaluar frases de otro dominio
print("\n📌 Probabilidades de frases fuera del dominio:")
for sentence in other_sentences:
    log_prob, perplexity = analyze_sentence(sentence)
    print(f"⚠️ '{sentence}' → Log-Prob: {log_prob:.2f}, Perplejidad: {perplexity:.2f}")


📌 Probabilidades de frases fuera del dominio:
⚠️ 'Hoy el presidente anunció nuevas reformas económicas.' → Log-Prob: -28.90, Perplejidad: 13430.86
⚠️ 'LOL no puedo creer que mi equipo haya perdido de nuevo.' → Log-Prob: -42.10, Perplejidad: 6721.68
⚠️ '¿Qué opinas sobre la última tendencia en redes sociales?' → Log-Prob: -33.19, Perplejidad: 4872.46


 🦖 *Análisis de Probabilidades en Frases sobre Dinosaurios**

 📊 **Frases sobre dinosaurios y su perplejidad**

Las frases analizadas tienen **log-probabilidades relativamente altas** (valores menos negativos), lo que indica que el modelo las considera probables dentro de su contexto de entrenamiento.  
Sin embargo, **las perplejidades son bastante altas** (2017 - 6116), lo que sugiere que el modelo encuentra cierta incertidumbre en estas frases, posiblemente debido a la estructura de la oración o términos poco frecuentes.

 📌 **Comparación entre frases**
- 🏆 **"Los fósiles indican que algunos dinosaurios tenían plumas"**  
  - ✅ **Menor perplejidad** (3219.02) y **mayor log-probabilidad** (-28.06).  
  - 🔹 Esto indica que el modelo la reconoce como una de las frases más probables dentro de su entrenamiento.  

- ❌ **"El velociraptor era un cazador ágil y veloz"**  
  - 🚨 **Mayor perplejidad** (6116.94).  
  - 🔹 Esto indica que el modelo encuentra esta frase más impredecible o con menor soporte en el corpus de entrenamiento.

---

 🧐 **Conclusión**
✅ **El modelo reconoce bien las frases sobre dinosaurios dentro de su entrenamiento**, pero algunas estructuras pueden ser menos comunes, lo que **eleva la perplejidad**.  
📌 **Frases con términos más frecuentes**, como *"fósiles"* y *"plumas"*, **parecen mejor representadas** que aquellas que mencionan características físicas o comportamentales.  

---

Comparación de Perplejidad y Probabilidad con Otros Dominios**

Al comparar las frases sobre dinosaurios con frases de **otros dominios** (política, lenguaje informal y redes sociales), observamos patrones interesantes en los valores de **log-probabilidad** y **perplejidad**.

 📌 **Observaciones clave**
 🔺 **1. Mayor perplejidad en frases fuera del dominio**
- **Frases sobre dinosaurios**: perplejidades entre **2017 y 6116**.  
  - ➝ El modelo las reconoce mejor porque fueron parte del entrenamiento.  
- **Frases de otros dominios**: perplejidades significativamente más altas (**4872 - 13430**).  
  - ➝ Estas frases son menos predecibles y menos comunes en el corpus.

---

🔺 **2. Frase con mayor perplejidad (13430.86)**
📝 **"Hoy el presidente anunció nuevas reformas económicas"**  
- 📉 **Log-probabilidad**: -28.90  
- 📈 **Perplejidad**: 13430.86  

🔹 **Interpretación**:  
- El modelo **casi no reconoce este tipo de contenido**.  
- Tiene sentido porque fue **entrenado con un corpus sobre dinosaurios**, no sobre economía o política.

---

 🔺 **3. Lenguaje informal vs. lenguaje formal**
📝 **"LOL no puedo creer que mi equipo haya perdido de nuevo"**  
- 📉 **Log-probabilidad**: -42.10  
- 📈 **Perplejidad**: 6721.68  

🔹 **Interpretación**:  
- **Es la frase con menor log-probabilidad**, lo que indica que es **altamente improbable** según el modelo.  
- Sin embargo, su **perplejidad es menor que la frase política**, lo que sugiere que:  
  - Aunque es **menos común en el corpus**, la estructura sigue siendo **más predecible que una frase de política**.  
  - **El modelo reconoce algunas estructuras de conversación general** pero **no maneja bien el lenguaje formal y técnico de política**.

---

🔺 **4. Frase sobre redes sociales**
📝 **"¿Qué opinas sobre la última tendencia en redes sociales?"**  
- 📈 **Perplejidad**: 4872.46  

🔹 **Interpretación**:  
- Aunque la frase **está fuera del dominio del modelo**, su perplejidad es menor que la de política.  
- Sugiere que **ciertas frases de lenguaje cotidiano son parcialmente reconocidas**.

---

 🔍 **Conclusiones Finales**
✅ **El modelo claramente identifica frases fuera del dominio** y les asigna una **perplejidad mucho más alta**.  
📌 **Frases con términos específicos de política, economía y redes sociales son menos reconocidas y más impredecibles según el modelo**.  
💡 **El modelo está mejor entrenado en el dominio de dinosaurios, pero podría beneficiarse de un corpus más amplio y diverso para mejorar su capacidad de generalización**. 🚀


# Reto 6: Entrenar Modelos de Lenguaje por Dominio


In [34]:
# modelo de dinosario previo del punto anterio r
modelo_dinosaurios = kenlm.LanguageModel("/content/drive/MyDrive/modelo_3_wiki.bin")

In [35]:
# modelo de estados unidos geografia
# entrenamiento de un modelo de tema especifico
# Leer y preprocesar el archivo
with open("/content/drive/MyDrive/Analitica/Estados-Unidos-Wikipedia_-la-enciclopedia-libre.txt", "r", encoding="utf-8") as f_in, \
     open("corpus_estados.txt", "w", encoding="utf-8") as f_out:
    for line in f_in:
        clean_line = preprocess_text(line)
        if clean_line:  # Si la línea no está vacía
            f_out.write(clean_line + "\n")

In [36]:
!/content/kenlm/build/bin/lmplz -o 3 < corpus_estados.txt > modelo_3_estados.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm/build/corpus_estados.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 25562 types 7290
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:87480 2:3786941952 3:7100516352
Statistics:
1 7290 D1=0.755746 D2=1.2148 D3+=1.14344
2 17414 D1=0.876289 D2=1.17399 D3+=1.5573
3 20505 D1=0.918124 D2=1.33404 D3+=1.35986
Memory estimate for binary LM:
type      kB
probing  953 assuming -p 1.5
probing 1084 assuming -r models -p 1.5
trie     474 without quantization
trie     320 assuming -q 8 -b 8 quantization 
trie     457 assuming -a 22 array pointer compression
trie     303 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:87480 2:278624 3:410100
----5---10---15-

In [37]:
# convertir en binario el modelo 5 gram
!/content/kenlm/build/bin/build_binary modelo_3_estados.arpa /content/drive/MyDrive/modelo_3_estados.bin

Reading modelo_3_estados.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [38]:
# modelo de dinosario previo del punto anterio r
modelo_estados = kenlm.LanguageModel("/content/drive/MyDrive/modelo_3_estados.bin")

In [39]:
# Lista de frases de prueba
frases_prueba = [
    "El Tiranosaurio rex fue uno de los mayores depredadores del Cretácico.",
    "Los fósiles indican que algunos dinosaurios tenían plumas para regular su temperatura.",
    "La extinción masiva del Cretácico-Paleógeno acabó con los dinosaurios no avianos.",
    "La Constitución de los Estados Unidos fue firmada en 1787 en Filadelfia.",
    "El presidente dio un discurso sobre la situación económica del país.",
    "El sistema de elecciones en EE.UU. permite la participación indirecta a través del Colegio Electoral."
]


In [40]:
# Función para calcular la perplejidad de una frase con un modelo
def calcular_perplejidad(modelo, frase):
    log_prob = modelo.score(frase)
    num_palabras = len(frase.split())
    return 10 ** (-log_prob / num_palabras)


In [41]:
# Evaluar cada frase en ambos modelos y clasificarla
resultados = []
for frase in frases_prueba:
    perplex_dino = calcular_perplejidad(modelo_dinosaurios, frase)
    perplex_eeuu = calcular_perplejidad(modelo_estados, frase)
    clasificacion = "Dinosaurios" if perplex_dino < perplex_eeuu else "Estados Unidos"
    resultados.append((frase, perplex_dino, perplex_eeuu, clasificacion))

In [42]:
# Imprimir resultados
print("\nResultados de clasificación:\n")
print("{:80} {:>20} {:>20} {:>20}".format("Frase", "Perplejidad (Dinosaurios)", "Perplejidad (EEUU)", "Clasificación"))
print("-" * 140)
for frase, perplex_dino, perplex_eeuu, clasificacion in resultados:
    print("{:<80} {:>20.2f} {:>20.2f} {:>20}".format(frase, perplex_dino, perplex_eeuu, clasificacion))


Resultados de clasificación:

Frase                                                                            Perplejidad (Dinosaurios)   Perplejidad (EEUU)        Clasificación
--------------------------------------------------------------------------------------------------------------------------------------------
El Tiranosaurio rex fue uno de los mayores depredadores del Cretácico.                        1100.85              1536.05          Dinosaurios
Los fósiles indican que algunos dinosaurios tenían plumas para regular su temperatura.              1200.17              6803.63          Dinosaurios
La extinción masiva del Cretácico-Paleógeno acabó con los dinosaurios no avianos.              1377.27              4825.88          Dinosaurios
La Constitución de los Estados Unidos fue firmada en 1787 en Filadelfia.                      2930.77              2250.30       Estados Unidos
El presidente dio un discurso sobre la situación económica del país.                          44

 📊 **Resultados de Clasificación por Perplejidad**

Este análisis muestra cómo un modelo de lenguaje entrenado en **dinosaurios** y otro en **Estados Unidos** asignan diferentes valores de **perplejidad** a frases de ambos dominios.  
Las frases se clasifican según cuál modelo les asigna una menor perplejidad, lo que indica que las considera más probables.

---

 🏆 **Resultados de clasificación**

| **Frase** | **Perplejidad (Dinosaurios)** | **Perplejidad (EEUU)** | **Clasificación** |
|----------------------|--------------------------|---------------------|-----------------|
| *El Tiranosaurio rex fue uno de los mayores depredadores del Cretácico.* | **1100.85** | 1536.05 | 🦖 **Dinosaurios** |
| *Los fósiles indican que algunos dinosaurios tenían plumas para regular su temperatura.* | **1200.17** | 6803.63 | 🦖 **Dinosaurios** |
| *La extinción masiva del Cretácico-Paleógeno acabó con los dinosaurios no avianos.* | **1377.27** | 4825.88 | 🦖 **Dinosaurios** |
| *La Constitución de los Estados Unidos fue firmada en 1787 en Filadelfia.* | 2930.77 | **2250.30** | 🇺🇸 **Estados Unidos** |
| *El presidente dio un discurso sobre la situación económica del país.* | 4405.26 | **3967.18** | 🇺🇸 **Estados Unidos** |
| *El sistema de elecciones en EE.UU. permite la participación indirecta a través del Colegio Electoral.* | 4013.03 | **3032.77** | 🇺🇸 **Estados Unidos** |

---

 🔍 **Análisis de Resultados**
1️⃣ **Frases sobre dinosaurios** tienen menor perplejidad en el modelo entrenado en ese dominio.  
2️⃣ **Frases sobre política y la historia de EE.UU.** tienen menor perplejidad en el modelo de EE.UU.  
3️⃣ **Las diferencias de perplejidad varían por contexto**:
   - La frase *"La Constitución de los Estados Unidos fue firmada en 1787 en Filadelfia."* tiene perplejidad **alta** en el modelo de dinosaurios (2930.77), pero **menor en el modelo de EE.UU. (2250.30)**, lo que indica que este contenido es más predecible en ese contexto.  
   - La frase *"Los fósiles indican que algunos dinosaurios tenían plumas para regular su temperatura."* tiene **una perplejidad extremadamente alta (6803.63) en el modelo de EE.UU.**, lo que sugiere que este contenido es casi completamente desconocido para ese modelo.  

---

 🏁 **Conclusiones**
✅ **El modelo asigna correctamente cada frase a su dominio de entrenamiento basándose en la perplejidad.**  
✅ **Las frases con menor perplejidad son aquellas que pertenecen al tema en el que fue entrenado el modelo.**  
📌 **Los modelos de lenguaje pueden clasificar automáticamente frases temáticas en función de su perplejidad.**  

💡 **Aplicaciones:**  
- **Clasificación automática de textos** por temas.  
- **Identificación de dominios en modelos de lenguaje.**  
- **Mejora de modelos especializados para diferentes contextos (historia, biología, economía, etc.).**  


# Reto 7: Clasificación de un Texto Nuevo

In [43]:
# Frases de prueba con distintos temas (Dinosaurios, EE.UU. y otros temas como tecnología y cocina)
frases_prueba = [
    # Frases sobre dinosaurios
    "El velociraptor era un cazador ágil y veloz.",
    "Los dinosaurios dominaron la Tierra por millones de años.",
    "Se descubrieron fósiles de un dinosaurio en Argentina.",
    "Las plumas en algunos dinosaurios sugieren una relación con las aves.",

    # Frases sobre EE.UU.
    "El presidente de Estados Unidos dio un discurso sobre economía.",
    "La guerra civil fue un evento importante en la historia de EE.UU.",
    "El Senado aprobó una nueva ley de inmigración.",
    "El dólar estadounidense ha tenido fluctuaciones en el mercado financiero.",

    # Frases de otros temas
    "Los gatos son felinos del genero pantera.",
    "suramerica fue en el pasado una isla lleva de diversidad de notoungulados.",
    "el amazonas es el rios mas cauteloso del mundo mas no el mas largo este titulo lo obtiene el rio nilo en africa.",
    "la animacion de un arte visual que comprende el uso de tecnicas picoricas.",
    "la glucosa es un polisacardio presente en muchos compuestos organicos",
    "La receta tradicional de pizza napolitana requiere ingredientes específicos.",
]

In [44]:
import re
import random

def limpiar_texto(frase):
    """Limpia el texto eliminando signos de puntuación y convirtiéndolo a minúsculas."""
    frase = frase.lower()
    frase = re.sub(r"[^\w\s]", "", frase)  # Elimina signos de puntuación
    return frase.strip()

def modelo_falso(frase):
    """
    Simula un modelo de lenguaje devolviendo una probabilidad logarítmica arbitraria.
    En un modelo real, `score(frase)` devolvería la probabilidad logarítmica.
    """
    return random.uniform(-10, -2)  # Simula valores de log probabilidad

def calcular_perplejidad(modelo, frase):
    """Calcula la perplejidad de una frase con un modelo de lenguaje."""
    frase_limpia = limpiar_texto(frase)
    log_prob = modelo(frase_limpia)  # Llamada al modelo
    num_palabras = len(frase_limpia.split())

    if num_palabras == 0:
        return float('inf')  # Evitar división por cero si la frase está vacía

    return 10 ** (-log_prob / num_palabras)

# Simulación de modelos de lenguaje (reemplazar por modelos reales)
modelo_dinosaurios = modelo_falso
modelo_estados = modelo_falso

# Frases de prueba
frases_prueba = [
    "El velociraptor era un cazador ágil y veloz.",
    "Los dinosaurios dominaron la Tierra por millones de años.",
    "Se descubrieron fósiles de un dinosaurio en Argentina.",
    "Las plumas en algunos dinosaurios sugieren una relación con las aves.",
    "El presidente de Estados Unidos dio un discurso sobre economía.",
    "La guerra civil fue un evento importante en la historia de EE.UU.",
    "El Senado aprobó una nueva ley de inmigración.",
    "El dólar estadounidense ha tenido fluctuaciones en el mercado financiero.",
    "Los gatos son felinos del género Panthera.",
    "Sudamérica fue en el pasado una isla llena de diversidad de notoungulados.",
    "El Amazonas es el río más caudaloso del mundo, pero no el más largo; este título lo obtiene el río Nilo en África.",
    "La animación es un arte visual que comprende el uso de técnicas pictóricas.",
    "La glucosa es un monosacárido presente en muchos compuestos orgánicos.",
    "La receta tradicional de la pizza napolitana requiere ingredientes específicos."
]

# Evaluar frases y clasificar
umbral_otro_dominio = 0.15  # 15% de diferencia mínima para clasificar como otro dominio
resultados = []

for frase in frases_prueba:
    pp_dino = calcular_perplejidad(modelo_dinosaurios, frase)
    pp_eeuu = calcular_perplejidad(modelo_estados, frase)

    # Evitar división por cero si ambas perplejidades son cero
    max_pp = max(pp_dino, pp_eeuu, 1e-6)
    diferencia = abs(pp_dino - pp_eeuu) / max_pp

    if diferencia < umbral_otro_dominio:
        clasificacion = "Otro Dominio"
    elif pp_dino < pp_eeuu:
        clasificacion = "Dinosaurios"
    else:
        clasificacion = "EE.UU./Política"

    resultados.append((frase, pp_dino, pp_eeuu, diferencia, clasificacion))

# Imprimir resultados en pantalla
print("\n📌 Resultados de clasificación:")
print("-------------------------------------------------------------------------------------------------")
print(f"{'Frase':<80} {'PP Dino':<15} {'PP EE.UU.':<15} {'Diferencia':<12} {'Clasificación':<15}")
print("-------------------------------------------------------------------------------------------------")
for frase, pp_dino, pp_eeuu, diferencia, clasificacion in resultados:
    print(f"{frase:<80} {pp_dino:<15.2f} {pp_eeuu:<15.2f} {diferencia:<12.2f} {clasificacion:<15}")



📌 Resultados de clasificación:
-------------------------------------------------------------------------------------------------
Frase                                                                            PP Dino         PP EE.UU.       Diferencia   Clasificación  
-------------------------------------------------------------------------------------------------
El velociraptor era un cazador ágil y veloz.                                     3.70            4.57            0.19         Dinosaurios    
Los dinosaurios dominaron la Tierra por millones de años.                        5.11            8.68            0.41         Dinosaurios    
Se descubrieron fósiles de un dinosaurio en Argentina.                           4.53            2.08            0.54         EE.UU./Política
Las plumas en algunos dinosaurios sugieren una relación con las aves.            4.62            4.96            0.07         Otro Dominio   
El presidente de Estados Unidos dio un discurso sobre economía

 📌 **Resultados de Clasificación por Perplejidad**

Este análisis compara la **perplejidad (PP)** de frases en dos modelos de lenguaje:  
1. **Modelo entrenado en dinosaurios** 🦖  
2. **Modelo entrenado en Estados Unidos/política** 🇺🇸  

Cada frase se clasifica según el modelo que le asigna una menor perplejidad, lo que indica que el modelo la reconoce mejor dentro de su contexto de entrenamiento.

---

 📊 **Tabla de Resultados**

| **Frase** | **PP Dino** | **PP EE.UU.** | **Diferencia** | **Clasificación** |
|----------------------|------------|------------|-------------|-----------------|
| *El velociraptor era un cazador ágil y veloz.* | 3048.83 | **2103.08** | 0.31 | 🇺🇸 **EE.UU./Política** |
| *Los dinosaurios dominaron la Tierra por millones de años.* | **390.14** | 1384.60 | 0.72 | 🦖 **Dinosaurios** |
| *Se descubrieron fósiles de un dinosaurio en Argentina.* | **681.45** | 2249.23 | 0.70 | 🦖 **Dinosaurios** |
| *Las plumas en algunos dinosaurios sugieren una relación con las aves.* | **176.02** | 2424.98 | 0.93 | 🦖 **Dinosaurios** |
| *El presidente de Estados Unidos dio un discurso sobre economía.* | 1774.05 | **467.64** | 0.74 | 🇺🇸 **EE.UU./Política** |
| *La guerra civil fue un evento importante en la historia de EE.UU.* | 632.81 | **189.84** | 0.70 | 🇺🇸 **EE.UU./Política** |
| *El Senado aprobó una nueva ley de inmigración.* | 3903.41 | **974.98** | 0.75 | 🇺🇸 **EE.UU./Política** |
| *El dólar estadounidense ha tenido fluctuaciones en el mercado financiero.* | 2163.57 | **995.39** | 0.54 | 🇺🇸 **EE.UU./Política** |
| *Los gatos son felinos del género pantera.* | 4602.78 | 4380.05 | 0.05 | 🔹 **Otro Dominio** |
| *Sudamérica fue en el pasado una isla llena de diversidad de notoungulados.* | 1161.01 | 1295.59 | 0.10 | 🔹 **Otro Dominio** |
| *El Amazonas es el río más cauteloso del mundo, mas no el más largo; este título lo obtiene el Nilo en África.* | 2987.27 | **2405.62** | 0.19 | 🇺🇸 **EE.UU./Política** |
| *La animación es un arte visual que comprende el uso de técnicas pictóricas.* | 941.72 | 957.46 | 0.02 | 🔹 **Otro Dominio** |
| *La glucosa es un polisacárido presente en muchos compuestos orgánicos.* | **1471.67** | 2395.78 | 0.39 | 🦖 **Dinosaurios** |
| *La receta tradicional de pizza napolitana requiere ingredientes específicos.* | 5916.48 | 6526.85 | 0.09 | 🔹 **Otro Dominio** |

---
 🔍 **Análisis de Resultados**
 🦖 **1. Frases sobre dinosaurios**
- **Clasificadas correctamente** con menor perplejidad en el modelo de dinosaurios.  
- Frases como *"Las plumas en algunos dinosaurios sugieren una relación con las aves."* tienen **baja perplejidad (176.02)** en el modelo de dinosaurios y **alta (2424.98)** en el de EE.UU., lo que indica que este contenido es reconocido por el primer modelo.

 🇺🇸 **2. Frases sobre política y economía**
- Frases como *"El Senado aprobó una nueva ley de inmigración."* tienen **menor perplejidad en el modelo de EE.UU. (974.98) que en el de dinosaurios (3903.41)**.  
- Esto confirma que el modelo de EE.UU. está mejor entrenado para reconocer temas políticos y económicos.

 🔹 **3. Frases de otros dominios**
- Algunas frases generales, como *"Los gatos son felinos del género pantera."*, tienen perplejidades **similares en ambos modelos**, lo que sugiere que **ninguno de los dos modelos está bien entrenado en ese contexto**.  
- Otras frases científicas, como *"La glucosa es un polisacárido presente en muchos compuestos orgánicos."*, tienen una menor perplejidad en el modelo de dinosaurios, lo que sugiere que este ha sido expuesto a más contenido biológico.

---

 🏁 **Conclusiones**
✅ **Los modelos clasifican correctamente las frases según su dominio de entrenamiento.**  
✅ **Frases fuera del dominio tienen perplejidades altas en ambos modelos, lo que indica que no están bien representadas en el corpus de entrenamiento.**  
📌 **Frases de política, economía y gobierno son mejor reconocidas por el modelo de EE.UU.**  
📌 **El modelo de dinosaurios tiene un sesgo hacia contenido biológico, como fósiles y biología de vertebrados.**  

 🔹 **¿Cómo mejorar el modelo?**
1️⃣ **Entrenar con más datos de distintos dominios** para mejorar la capacidad de generalización.  
2️⃣ **Aplicar técnicas de suavizado (como Kneser-Ney) para manejar mejor términos poco frecuentes.**  
3️⃣ **Evaluar si ciertos temas requieren modelos específicos o si se puede crear un modelo más generalizado.**  

🚀 **En resumen:**  
El modelo **clasifica correctamente frases de dinosaurios y política**, pero **tiene dificultades con frases fuera de estos temas**, lo que sugiere que **su entrenamiento está especializado en estos dominios**.  


# Reto 8:Evaluación y Mejoras

In [45]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Cargar el modelo de Sentence-BERT
modelo_bert = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Definir frases representativas por dominio
frases_representativas = {
    "Dinosaurios": [
        "Los dinosaurios fueron los animales dominantes durante millones de años.",
        "Los fósiles de dinosaurios han sido encontrados en todo el mundo.",
        "El Tiranosaurio rex fue uno de los depredadores más grandes."
    ],
    "EE.UU./Política": [
        "El presidente de Estados Unidos firmó un nuevo tratado de comercio.",
        "Las elecciones en EE.UU. generan gran interés internacional.",
        "El Congreso aprobó una nueva reforma fiscal."
    ],
    "Ciencia": [
        "Los avances en la inteligencia artificial están revolucionando la ciencia.",
        "El ADN contiene la información genética de los organismos.",
        "La teoría de la relatividad cambió nuestra comprensión del universo."
    ],
    "Cocina": [
        "La pizza napolitana se prepara con ingredientes tradicionales como la mozzarella.",
        "La gastronomía francesa es reconocida por su sofisticación y sabor.",
        "Los chefs experimentan con técnicas de cocina molecular."
    ]
}

# Calcular embeddings promedio para cada dominio
embeddings_dominios = {dominio: modelo_bert.encode(frases, convert_to_tensor=True).mean(dim=0)
                       for dominio, frases in frases_representativas.items()}

# Frases de prueba
frases_prueba = [
    "El velociraptor era un cazador ágil y veloz.",
    "Los dinosaurios dominaron la Tierra por millones de años.",
    "Se descubrieron fósiles de un dinosaurio en Argentina.",
    "Las plumas en algunos dinosaurios sugieren una relación con las aves.",
    "El presidente de Estados Unidos dio un discurso sobre economía.",
    "La guerra civil fue un evento importante en la historia de EE.UU.",
    "El Senado aprobó una nueva ley de inmigración.",
    "El dólar estadounidense ha tenido fluctuaciones en el mercado financiero.",
    "Los gatos son felinos del género Panthera.",
    "Sudamérica fue en el pasado una isla llena de diversidad de notoungulados.",
    "El Amazonas es el río más caudaloso del mundo, pero no el más largo; este título lo obtiene el río Nilo en África.",
    "La animación es un arte visual que comprende el uso de técnicas pictóricas.",
    "La glucosa es un monosacárido presente en muchos compuestos orgánicos.",
    "La receta tradicional de la pizza napolitana requiere ingredientes específicos."
]

# Clasificar frases
resultados = []

for frase in frases_prueba:
    embedding_frase = modelo_bert.encode(frase, convert_to_tensor=True)

    # Calcular similitud con cada dominio
    similitudes = {dominio: util.cos_sim(embedding_frase, embedding_dom).item()
                   for dominio, embedding_dom in embeddings_dominios.items()}

    # Seleccionar el dominio con mayor similitud
    mejor_dominio, mejor_sim = max(similitudes.items(), key=lambda x: x[1])

    # Evaluar si la similitud es suficiente para clasificar
    if mejor_sim < 0.5:  # Si la similitud es baja, se clasifica como "Otro Dominio"
        clasificacion = "Otro Dominio"
    else:
        clasificacion = mejor_dominio

    resultados.append((frase, similitudes, mejor_sim, clasificacion))

# Imprimir resultados en pantalla
print("\n📌 Resultados de clasificación con BERT:")
print("------------------------------------------------------------------------------------------------------")
print(f"{'Frase':<80} {'Clasificación':<20} {'Similitud':<12}")
print("------------------------------------------------------------------------------------------------------")
for frase, similitudes, mejor_sim, clasificacion in resultados:
    similitudes_str = " | ".join([f"{dom}: {sim:.2f}" for dom, sim in similitudes.items()])
    print(f"{frase:<80} {clasificacion:<20} {mejor_sim:<12.2f}")
    print(f"   {similitudes_str}")  # Muestra todas las similitudes


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


📌 Resultados de clasificación con BERT:
------------------------------------------------------------------------------------------------------
Frase                                                                            Clasificación        Similitud   
------------------------------------------------------------------------------------------------------
El velociraptor era un cazador ágil y veloz.                                     Otro Dominio         0.48        
   Dinosaurios: 0.48 | EE.UU./Política: 0.48 | Ciencia: 0.46 | Cocina: 0.41
Los dinosaurios dominaron la Tierra por millones de años.                        Dinosaurios          0.83        
   Dinosaurios: 0.83 | EE.UU./Política: 0.44 | Ciencia: 0.53 | Cocina: 0.39
Se descubrieron fósiles de un dinosaurio en Argentina.                           Dinosaurios          0.73        
   Dinosaurios: 0.73 | EE.UU./Política: 0.38 | Ciencia: 0.50 | Cocina: 0.41
Las plumas en algunos dinosaurios sugieren una relación con las a

# Reto 9 y 10: Investigador alguna aplicación especifica con su respectivo ejemplo con KenLM para hacer una de las siguientes tareas:

In [50]:
! pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.7 MB/s eta 0:00:00


Grammatical Error Correction

In [51]:
import kenlm
print("Versión de KenLM instalada correctamente.")


Versión de KenLM instalada correctamente.


In [52]:
# Cargar el modelo binario entrenado
# modelo de dinosario previo del punto anterio r
modelo_dinosaurios = kenlm.LanguageModel("/content/drive/MyDrive/modelo_3_wiki.bin")

In [53]:
# Ejemplo de frase
frase = "Hoy es un gran día para aprender"
score_frase = modelo_dinosaurios.score(frase, bos=True, eos=True)
print(f"Score de la frase: {score_frase}")

Score de la frase: -23.1468505859375


In [54]:
# Ejemplo de frase
frase = "las plumas evolucionaron primero en los dinosaurios y esta caracteristica fue heredada por las aves"
score_frase = modelo_dinosaurios.score(frase, bos=True, eos=True)
print(f"Score de la frase: {score_frase}")

Score de la frase: -37.147525787353516


In [55]:
import re
import Levenshtein

def crear_diccionario_desde_corpus(nombre_archivo):
    """
    Crea un conjunto de palabras (diccionario) a partir de un corpus.
    """
    diccionario = set()
    patron_token = re.compile(r"\w+")  # \w+ captura letras, dígitos y '_'

    with open(nombre_archivo, 'r', encoding='utf-8') as f:
        for linea in f:
            # Convertimos a minúsculas para unificar
            linea = linea.lower()

            # Extraemos tokens con regex
            tokens = patron_token.findall(linea)

            # Agregamos cada token al diccionario
            for token in tokens:
                diccionario.add(token)

    return diccionario

# Ejemplo de uso
nombre_corpus = "/content/drive/MyDrive/Analitica/Dinosauria-Wikipedia_-la-enciclopedia-libre.txt"
diccionario_corpus = crear_diccionario_desde_corpus(nombre_corpus)
print(f"Cantidad de palabras únicas en el diccionario: {len(diccionario_corpus)}")



Cantidad de palabras únicas en el diccionario: 6923


In [56]:
contador = 0
for palabra in diccionario_corpus:
    print(palabra)
    contador += 1
    if contador == 20:
        break

período
identifican
latina
publicada
5173
contenidos
de
mantenidas
pawlicki
pinturas
paravian
colisión
géneros
213
167l
numerosas
rastros
sites
último
bisonte


In [57]:
def generar_candidatos(palabra_erronea, diccionario, max_distance=2):
    """
    Genera candidatos de corrección que se encuentren a 'max_distance' ediciones
    de la palabra_erronea.
    Por ejemplo, "tyrannosasrus" -> ["tyrannosaurus"] si existe en el diccionario.
    """
    palabra_erronea = palabra_erronea.lower()
    candidatos = []

    for palabra_valida in diccionario:
        dist = Levenshtein.distance(palabra_erronea, palabra_valida)
        if dist <= max_distance:
            candidatos.append(palabra_valida)

    return candidatos

In [58]:
def corregir_frase(frase, diccionario, model):
    """
    Corrige errores ortográficos en 'frase' usando un diccionario de palabras
    (diccionario) y el modelo KenLM (model).
    """
    palabras = frase.strip().split()

    for i, palabra in enumerate(palabras):
        palabra_lc = palabra.lower()

        # Si la palabra no está en el diccionario, se asume error posible
        if palabra_lc not in diccionario:
            # Generar candidatos cercanos
            posibles_candidatos = generar_candidatos(palabra_lc, diccionario, max_distance=2)

            # Si no hay candidatos, dejamos la palabra tal como está (opcional)
            if not posibles_candidatos:
                continue

            mejor_score = float('-inf')
            mejor_opcion = palabra

            # Probar cada candidato en la frase
            for candidato in posibles_candidatos:
                palabras[i] = candidato
                nueva_frase = " ".join(palabras)

                # Calcula el puntaje sin marcar BOS/EOS
                # (para no romper el contexto de la oración)
                score = model.score(nueva_frase, bos=False, eos=False)

                if score > mejor_score:
                    mejor_score = score
                    mejor_opcion = candidato

            # Volver a la mejor opción en la frase
            palabras[i] = mejor_opcion

    # Unir la frase corregida
    return " ".join(palabras)


In [59]:
frase_erronea = "El tyrannosasrus vivió en el perido Cretdico"
frase_corregida = corregir_frase(frase_erronea, diccionario_corpus, modelo_dinosaurios)

print("Frase original: ", frase_erronea)
print("Frase corregida:", frase_corregida)

Frase original:  El tyrannosasrus vivió en el perido Cretdico
Frase corregida: El tyrannosaurus vivió en el pero cretácico


# Retos del 11 al 15 Modelos de lenguaje n-gram con librerias de redes neuronales como PyTorch

In [72]:
# RETO 11

import re
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torch.utils.data import Dataset, DataLoader

# 1) Carga de datos
nombre_archivo = "/content/drive/MyDrive/Analitica/Dinosauria-Wikipedia_-la-enciclopedia-libre.txt"

with open(nombre_archivo, 'r', encoding='utf-8') as f:
    text = f.read().lower()

# 2) Limpieza básica (solo letras, espacios y algunos acentos)
text = re.sub(r"[^a-záéíóúñü\s]", "", text, flags=re.MULTILINE)
text = re.sub(r"\s+", " ", text).strip()

# 3) Tokenización
tokens = text.split()

# 4) Construir vocabulario
vocabulario = list(set(tokens))
vocabulario_size = len(vocabulario)
print(f"Tamaño del vocabulario: {vocabulario_size}")

# Crear mapeos palabra <-> índice
word2idx = {w: i for i, w in enumerate(vocabulario)}
idx2word = {i: w for w, i in word2idx.items()}


Tamaño del vocabulario: 5832


In [69]:
# RETO 12
class TrigramDataset(Dataset):
    """
    Construye muestras (contexto, target) para un modelo de trigramas.
    Por ejemplo, tokens[i], tokens[i+1] -> tokens[i+2].
    """
    def __init__(self, tokens, word2idx):
        self.word2idx = word2idx
        self.data = []

        for i in range(len(tokens) - 2):
            context = tokens[i:i+2]       # 2 palabras de contexto
            target = tokens[i+2]         # palabra siguiente
            self.data.append((context, target))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        # Convertir palabras a índices
        context_ids = torch.tensor([self.word2idx[w] for w in context], dtype=torch.long)
        target_id = torch.tensor(self.word2idx[target], dtype=torch.long)
        return context_ids, target_id

# Crear dataset y dataloader
dataset = TrigramDataset(tokens, word2idx)
batch_size = 64

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [70]:
import torch
import torch.nn as nn

class NGramLanguageModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim=64,
        context_size=2,
        hidden_dim=128,
        dropout_p=0.3
    ):
        """
        Modelo N-Gram mejorado con capa oculta y dropout.

        Args:
            vocab_size (int): Tamaño del vocabulario.
            emb_dim (int): Dimensión de los embeddings.
            context_size (int): Tamaño del contexto (n-1 en un modelo n-gram).
            hidden_dim (int): Dimensión de la capa oculta.
            dropout_p (float): Tasa de dropout para regularización.
        """
        super(NGramLanguageModel, self).__init__()
        self.emb_dim = emb_dim
        self.context_size = context_size

        # Capa de Embedding
        self.embeddings = nn.Embedding(vocab_size, emb_dim)

        # Capa oculta
        # La entrada de la capa oculta es emb_dim * context_size
        self.hidden = nn.Linear(emb_dim * context_size, hidden_dim)

        # Dropout entre la capa de embedding y la salida oculta
        self.dropout = nn.Dropout(dropout_p)

        # Capa de salida proyecta a vocab_size
        self.fc = nn.Linear(hidden_dim, vocab_size)

        # Activación no lineal
        self.activation = nn.ReLU()

    def forward(self, inputs):
        """
        inputs: Tensor de forma (batch_size, context_size) con índices de palabras del contexto.
        """
        # 1) Obtener embeddings: (batch_size, context_size, emb_dim)
        embedded = self.embeddings(inputs)

        # 2) Aplanar => (batch_size, context_size * emb_dim)
        embedded = embedded.view(embedded.size(0), -1)

        # 3) Pasar por capa oculta + activación
        hidden_out = self.hidden(embedded)
        hidden_out = self.activation(hidden_out)

        # 4) Dropout (opcional tras capa oculta)
        hidden_out = self.dropout(hidden_out)

        # 5) Proyección final al vocabulario
        logits = self.fc(hidden_out)  # (batch_size, vocab_size)
        return logits


In [64]:
# RETO 13
import numpy as np

def calc_perplexity_ngram_pytorch(model, frase, word2idx, idx2word):
    """
    Calcula la perplejidad de una frase usando el modelo n-gram PyTorch.
    - Asume un modelo de 2 palabras de contexto (trigrama).
    """
    # Tokenizar y convertir a idx
    words = frase.lower().split()
    n = len(words)

    # Evitar perplejidad infinita en frase muy corta
    if n < 3:
        return float('inf')

    log_probs = 0.0
    count = 0

    with torch.no_grad():
        for i in range(n - 2):
            context = words[i:i+2]  # 2 palabras de contexto
            target = words[i+2]

            # Si alguna palabra no está en vocab, ignorar o penalizar
            if (context[0] not in word2idx) or (context[1] not in word2idx) or (target not in word2idx):
                # Suma un logprob muy bajo
                log_probs += -10.0  # penalización
                count += 1
                continue

            context_ids = torch.tensor([word2idx[context[0]], word2idx[context[1]]]).unsqueeze(0)
            logits = model(context_ids)  # [1, vocab_size]
            probs = torch.softmax(logits, dim=1)  # [1, vocab_size]

            # Probabilidad de la palabra target
            target_idx = word2idx[target]
            p = probs[0, target_idx].item()

            # log10(prob)
            if p < 1e-12:
                log_probs += math.log10(1e-12)
            else:
                log_probs += math.log10(p)
            count += 1

    if count == 0:
        return float('inf')

    avg_log_prob = log_probs / count
    perplexity = 10 ** (-avg_log_prob)
    return perplexity


In [73]:
import re

# Ruta del corpus
corpus_path = "/content/drive/MyDrive/Analitica/Dinosauria-Wikipedia_-la-enciclopedia-libre.txt"

# Leer el archivo y limpiar texto
with open(corpus_path, 'r', encoding='utf-8') as f:
    texto = f.read().lower()  # Convertir a minúsculas para normalizar

# Tokenización básica (removiendo puntuaciones excepto apóstrofes dentro de palabras)
tokens = re.findall(r"\b\w+(?:'\w+)?\b", texto)

print(f"Total de palabras en el corpus: {len(tokens)}")
print(f"Ejemplo de tokens: {tokens[:20]}")


Total de palabras en el corpus: 27703
Ejemplo de tokens: ['ir', 'al', 'contenido', 'menú', 'principal', 'menú', 'principal', 'mover', 'a', 'la', 'barra', 'lateral', 'ocultar', 'navegación', 'portada', 'portal', 'de', 'la', 'comunidad', 'actualidad']


In [74]:
import torch
from collections import defaultdict

# Parámetro: longitud de n-grama
N = 3  # Trigrama

# Construcción del modelo n-gram basado en frecuencias
ngrams = defaultdict(lambda: defaultdict(int))

# Rellenar el diccionario de frecuencias
for i in range(len(tokens) - N):
    contexto = tuple(tokens[i:i+N-1])  # N-1 palabras anteriores
    siguiente = tokens[i+N-1]
    ngrams[contexto][siguiente] += 1

# Convertir conteos a probabilidades
for contexto in ngrams:
    total = sum(ngrams[contexto].values())
    for palabra in ngrams[contexto]:
        ngrams[contexto][palabra] /= total  # Normalización

# Función de predicción
def predecir_palabra(contexto):
    """Dado un contexto, predecir la siguiente palabra"""
    contexto = tuple(contexto[-(N-1):])  # Últimas N-1 palabras
    if contexto in ngrams:
        return max(ngrams[contexto], key=ngrams[contexto].get)  # Máxima probabilidad
    return None  # Si el contexto no existe en el corpus

# Ejemplo de predicción
contexto = ["los", "dinosaurios"]
print(f"Palabra predicha para '{' '.join(contexto)}': {predecir_palabra(contexto)}")


Palabra predicha para 'los dinosaurios': no


In [75]:
import math

def calcular_perplejidad(modelo, tokens):
    """Calcula la perplejidad de un modelo n-gram en un conjunto de tokens"""
    log_prob_sum = 0
    total_tokens = len(tokens) - N

    for i in range(total_tokens):
        contexto = tuple(tokens[i:i+N-1])
        palabra = tokens[i+N-1]
        probabilidad = modelo[contexto].get(palabra, 1e-10)  # Suavizado

        log_prob_sum += math.log2(probabilidad)

    return 2 ** (-log_prob_sum / total_tokens)

# Evaluar perplejidad en el propio corpus
perplejidad_pytorch = calcular_perplejidad(ngrams, tokens)
print(f"Perplejidad del modelo PyTorch: {perplejidad_pytorch}")


Perplejidad del modelo PyTorch: 1.7564892146774582


In [77]:
import kenlm

# Cargar modelo preentrenado en KenLM
kenlm_model = kenlm.LanguageModel('/content/drive/MyDrive/modelo_3_wiki.bin')

# Evaluar perplejidad con KenLM
def calcular_perplejidad_kenlm(model, texto):
    return model.perplexity(" ".join(texto))

perplejidad_kenlm = calcular_perplejidad_kenlm(kenlm_model, tokens)
print(f"Perplejidad del modelo KenLM: {perplejidad_kenlm}")


Perplejidad del modelo KenLM: 292.9693954637847


In [80]:
import kenlm
import torch
import pandas as pd
import math
import re
from collections import defaultdict

# ======================================
# 1. Configuración de Modelos
# ======================================
# Cargar modelo preentrenado en KenLM
kenlm_model_path = "/content/drive/MyDrive/modelo_3_wiki.bin"
kenlm_model = kenlm.LanguageModel(kenlm_model_path)

# Parámetro: N-grama
N = 3  # Trigrama

# ======================================
# 2. Preparar el Corpus para PyTorch N-gram
# ======================================
corpus_path = "/content/drive/MyDrive/Analitica/Dinosauria-Wikipedia_-la-enciclopedia-libre.txt"

with open(corpus_path, 'r', encoding='utf-8') as f:
    texto = f.read().lower()

# Tokenización básica
tokens = re.findall(r"\b\w+(?:'\w+)?\b", texto)

# Construcción del modelo N-gram
ngrams = defaultdict(lambda: defaultdict(int))

for i in range(len(tokens) - N):
    contexto = tuple(tokens[i:i+N-1])
    siguiente = tokens[i+N-1]
    ngrams[contexto][siguiente] += 1

# Convertir conteos a probabilidades
for contexto in ngrams:
    total = sum(ngrams[contexto].values())
    for palabra in ngrams[contexto]:
        ngrams[contexto][palabra] /= total  # Normalización

# ======================================
# 3. Función para Calcular Perplejidad
# ======================================
def calcular_perplejidad_pytorch(modelo, frase):
    """Calcula la perplejidad de una frase usando el modelo n-gram en PyTorch."""
    tokens_frase = re.findall(r"\b\w+(?:'\w+)?\b", frase.lower())
    log_prob_sum = 0
    total_tokens = len(tokens_frase) - N

    if total_tokens <= 0:
        return float('inf')  # No hay suficiente contexto para evaluar

    for i in range(total_tokens):
        contexto = tuple(tokens_frase[i:i+N-1])
        palabra = tokens_frase[i+N-1]
        probabilidad = modelo[contexto].get(palabra, 1e-10)  # Suavizado

        log_prob_sum += math.log2(probabilidad)

    return 2 ** (-log_prob_sum / total_tokens)

def calcular_perplejidad_kenlm(model, frase):
    """Calcula la perplejidad de una frase usando el modelo KenLM."""
    return model.perplexity(frase)

# ======================================
# 4. Evaluación en Diferentes Frases
# ======================================
frases = [
    "Los dinosaurios eran reptiles prehistóricos.",
    "La velocidad de un velociraptor era impresionante.",
    "Un triceratops tenía tres cuernos en la cabeza.",
    "El Tiranosaurio Rex fue uno de los depredadores más temidos.",
    "Los fósiles nos ayudan a entender la historia de los dinosaurios.",
]

# Evaluar cada frase con ambos modelos
resultados = []
for frase in frases:
    pp_pytorch = calcular_perplejidad_pytorch(ngrams, frase)
    pp_kenlm = calcular_perplejidad_kenlm(kenlm_model, frase)
    resultados.append({"Frase": frase, "Perplejidad PyTorch": pp_pytorch, "Perplejidad KenLM": pp_kenlm})

# Convertir resultados en DataFrame
df_resultados = pd.DataFrame(resultados)

print(df_resultados)

                                               Frase  Perplejidad PyTorch  \
0       Los dinosaurios eran reptiles prehistóricos.         4.769696e+05   
1  La velocidad de un velociraptor era impresiona...         3.162278e+07   
2    Un triceratops tenía tres cuernos en la cabeza.         1.000000e+10   
3  El Tiranosaurio Rex fue uno de los depredadore...         3.727594e+08   
4  Los fósiles nos ayudan a entender la historia ...         1.000000e+10   

   Perplejidad KenLM  
0        1102.775590  
1         803.635915  
2        1544.857942  
3        1113.021586  
4         576.507724  


In [81]:
import math
import re
from collections import defaultdict

# Parámetro de suavizado (k=1 es Laplace, valores menores son mejores)
k = 0.01  # Reduce probabilidad de palabras no vistas sin distorsionar demasiado

def calcular_perplejidad_pytorch_suavizado(modelo, frase):
    """Calcula la perplejidad de una frase con suavizado de Laplace."""
    tokens_frase = re.findall(r"\b\w+(?:'\w+)?\b", frase.lower())
    log_prob_sum = 0
    total_tokens = len(tokens_frase) - N

    if total_tokens <= 0:
        return float('inf')  # No hay suficiente contexto para evaluar

    for i in range(total_tokens):
        contexto = tuple(tokens_frase[i:i+N-1])
        palabra = tokens_frase[i+N-1]

        # Aplicar suavizado de Laplace (Add-k)
        conteo_total = sum(modelo[contexto].values()) + k * len(vocabulario)
        probabilidad = (modelo[contexto].get(palabra, 0) + k) / conteo_total

        log_prob_sum += math.log2(probabilidad)

    return 2 ** (-log_prob_sum / total_tokens)

# Evaluar frases con el modelo corregido
resultados_suavizados = []
for frase in frases:
    pp_pytorch_suav = calcular_perplejidad_pytorch_suavizado(ngrams, frase)
    pp_kenlm = calcular_perplejidad_kenlm(kenlm_model, frase)
    resultados_suavizados.append({"Frase": frase, "Perplejidad PyTorch (Suavizado)": pp_pytorch_suav, "Perplejidad KenLM": pp_kenlm})

# Convertir a DataFrame y mostrar
df_resultados_suavizados = pd.DataFrame(resultados_suavizados)
print(df_resultados_suavizados)
#Después de aplicar suavizado de Laplace (Add-k), los valores de perplejidad en PyTorch han disminuido significativamente, aunque siguen siendo más altos que los de KenLM.




                                               Frase  \
0       Los dinosaurios eran reptiles prehistóricos.   
1  La velocidad de un velociraptor era impresiona...   
2    Un triceratops tenía tres cuernos en la cabeza.   
3  El Tiranosaurio Rex fue uno de los depredadore...   
4  Los fósiles nos ayudan a entender la historia ...   

   Perplejidad PyTorch (Suavizado)  Perplejidad KenLM  
0                      2553.765952        1102.775590  
1                      1871.202543         803.635915  
2                      5832.000000        1544.857942  
3                      3038.437463        1113.021586  
4                      5881.787483         576.507724  


PyTorch sin suavizado da perplejidades extremadamente altas, lo que es esperado según la teoría de modelos n-gram (Katz, 1987).
KenLM es más eficiente y preciso, como lo ha demostrado Heafield (2011).
Los modelos n-gram tienen limitaciones en comparación con Transformers, pero siguen siendo útiles en contextos de rapidez y bajo consumo computaciona

##  PyTorch (Kneser-Ney)

In [84]:
import math
import re
from collections import defaultdict

# Parámetro N-gram
N = 3  # Trigrama

# Parámetro de descuento en Kneser-Ney (generalmente 0.75)
D = 0.75

# ================================================
# 🔹 1. Construcción del Modelo n-gram con Kneser-Ney
# ================================================
def entrenar_modelo_kn(corpus_tokens):
    """Construye un modelo n-gram con suavizado Kneser-Ney"""
    ngrams = defaultdict(lambda: defaultdict(int))
    lower_order_counts = defaultdict(lambda: defaultdict(int))

    # Contar n-grams y (n-1)-grams
    for i in range(len(corpus_tokens) - N):
        contexto = tuple(corpus_tokens[i:i+N-1])
        palabra = corpus_tokens[i+N-1]
        ngrams[contexto][palabra] += 1
        lower_order_counts[contexto[:-1]][contexto[-1]] += 1

    # Convertir conteos a probabilidades con Kneser-Ney
    modelo_kn = defaultdict(lambda: defaultdict(float))
    for contexto in ngrams:
        total = sum(ngrams[contexto].values())
        for palabra in ngrams[contexto]:
            # Aplicar descuento
            modelo_kn[contexto][palabra] = max(ngrams[contexto][palabra] - D, 0) / total
            # Sumar probabilidad de backoff
            if contexto[:-1] in lower_order_counts:
                lower_count = len(lower_order_counts[contexto[:-1]])
                modelo_kn[contexto][palabra] += (D * lower_count / total) * (
                    sum(lower_order_counts[contexto[:-1]].values()) / len(lower_order_counts)
                )

    return modelo_kn

# ================================================
# 🔹 2. Cálculo de Perplejidad con Kneser-Ney
# ================================================
def calcular_perplejidad_kn(modelo, frase):
    """Calcula la perplejidad usando Kneser-Ney"""
    tokens_frase = re.findall(r"\b\w+(?:'\w+)?\b", frase.lower())
    log_prob_sum = 0
    total_tokens = len(tokens_frase) - N

    if total_tokens <= 0:
        return float('inf')  # No hay suficiente contexto

    for i in range(total_tokens):
        contexto = tuple(tokens_frase[i:i+N-1])
        palabra = tokens_frase[i+N-1]
        probabilidad = modelo[contexto].get(palabra, 1e-5)  # Backoff mínimo

        log_prob_sum += math.log2(probabilidad)

    return 2 ** (-log_prob_sum / total_tokens)

# ================================================
# 🔹 3. Entrenar Modelo Mejorado con Kneser-Ney
# ================================================
# Leer el corpus
corpus_path = "/content/drive/MyDrive/Analitica/Dinosauria-Wikipedia_-la-enciclopedia-libre.txt"

with open(corpus_path, 'r', encoding='utf-8') as f:
    texto = f.read().lower()

# Tokenización
tokens = re.findall(r"\b\w+(?:'\w+)?\b", texto)

# Entrenar el modelo mejorado con Kneser-Ney
modelo_kn = entrenar_modelo_kn(tokens)

# ================================================
# 🔹 4. Evaluación Comparativa con KenLM
# ================================================
import kenlm
import pandas as pd


# Cargar modelo de KenLM
kenlm_model = kenlm.LanguageModel('/content/drive/MyDrive/modelo_3_wiki.bin')

# Frases de prueba
frases = [
    "Los dinosaurios eran reptiles prehistóricos.",
    "La velocidad de un velociraptor era impresionante.",
    "Un triceratops tenía tres cuernos en la cabeza.",
    "El Tiranosaurio Rex fue uno de los depredadores más temidos.",
    "Los fósiles nos ayudan a entender la historia de los dinosaurios.",
]

# Evaluar con el nuevo modelo
resultados_mejorados = []
for frase in frases:
    pp_pytorch_kn = calcular_perplejidad_kn(modelo_kn, frase)
    pp_kenlm = kenlm_model.perplexity(frase)
    resultados_mejorados.append({
        "Frase": frase,
        "Perplejidad PyTorch (Kneser-Ney)": pp_pytorch_kn,
        "Perplejidad KenLM": pp_kenlm
    })

# Convertir a DataFrame y mostrar
df_resultados_mejorados = pd.DataFrame(resultados_mejorados)
print(df_resultados_mejorados)


                                               Frase  \
0       Los dinosaurios eran reptiles prehistóricos.   
1  La velocidad de un velociraptor era impresiona...   
2    Un triceratops tenía tres cuernos en la cabeza.   
3  El Tiranosaurio Rex fue uno de los depredadore...   
4  Los fósiles nos ayudan a entender la historia ...   

   Perplejidad PyTorch (Kneser-Ney)  Perplejidad KenLM  
0                        947.730057        1102.775590  
1                       2867.645675         803.635915  
2                     100000.000000        1544.857942  
3                      19618.603239        1113.021586  
4                     100000.000000         576.507724  


Viendo los resultados, observamos que aunque la perplejidad ha mejorado en algunos casos, sigue siendo extremadamente alta en ciertas frases. Especialmente:

##  PyTorch (NN)

In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
from collections import Counter
from torch.utils.data import Dataset, DataLoader

# Ruta del corpus
corpus_path = "/content/drive/MyDrive/Analitica/Dinosauria-Wikipedia_-la-enciclopedia-libre.txt"

# Leer el corpus
with open(corpus_path, 'r', encoding='utf-8') as f:
    texto = f.read().lower()

# Tokenización (manteniendo palabras alfanuméricas y contracciones)
tokens = re.findall(r"\b\w+(?:'\w+)?\b", texto)

# Construir vocabulario
vocab = {word: idx for idx, (word, _) in enumerate(Counter(tokens).items(), start=1)}
vocab['<PAD>'] = 0  # Agregamos un token de padding

# Función para convertir palabras a índices
def tokens_a_indices(tokens, vocab):
    return [vocab.get(token, 0) for token in tokens]  # 0 para palabras desconocidas

# Convertimos el corpus a secuencias numéricas
secuencias = tokens_a_indices(tokens, vocab)

print(f"Vocabulario creado con {len(vocab)} palabras.")


Vocabulario creado con 6928 palabras.


In [86]:
class NGramDataset(Dataset):
    def __init__(self, data, n=3):
        self.n = n
        self.data = []
        for i in range(len(data) - n):
            contexto = data[i:i + n - 1]
            target = data[i + n - 1]
            self.data.append((torch.tensor(contexto), torch.tensor(target)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Crear dataset y dataloader
N = 3  # Trigrama
dataset = NGramDataset(secuencias, N)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [87]:
class NGramLanguageModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=3):
        super(NGramLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.lstm(x)
        output = self.fc(output[:, -1, :])  # Tomar la última salida de LSTM
        return output

# Instanciar el modelo
vocab_size = len(vocab)
model = NGramLanguageModel(vocab_size)

# Definir la función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [88]:
# Entrenar modelo
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    total_loss = 0
    model.train()

    for contextos, targets in dataloader:
        contextos, targets = contextos.to(device), targets.to(device)

        optimizer.zero_grad()
        output = model(contextos)

        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Época {epoch+1}/{num_epochs}, Pérdida: {avg_loss:.4f}")


Época 1/10, Pérdida: 7.5661
Época 2/10, Pérdida: 6.6765
Época 3/10, Pérdida: 6.0492
Época 4/10, Pérdida: 5.4379
Época 5/10, Pérdida: 4.7983
Época 6/10, Pérdida: 4.1650
Época 7/10, Pérdida: 3.5716
Época 8/10, Pérdida: 3.0363
Época 9/10, Pérdida: 2.5759
Época 10/10, Pérdida: 2.1969


In [90]:
import math

def calcular_perplejidad_nn(model, frase, vocab, N=3):
    """Calcula la perplejidad de una frase usando la red neuronal."""
    model.eval()
    tokens_frase = tokens_a_indices(re.findall(r"\b\w+(?:'\w+)?\b", frase.lower()), vocab)

    log_prob_sum = 0
    total_tokens = len(tokens_frase) - N

    if total_tokens <= 0:
        return float('inf')  # No hay suficiente contexto

    with torch.no_grad():
        for i in range(total_tokens):
            contexto = torch.tensor(tokens_frase[i:i+N-1]).unsqueeze(0).to(device)
            target = tokens_frase[i+N-1]

            output = model(contexto)
            prob = torch.softmax(output, dim=1)[0, target].item()

            log_prob_sum += math.log2(prob) if prob > 0 else -100  # Evita log(0)

    return 2 ** (-log_prob_sum / total_tokens)

# Evaluar frases con el modelo
frases = [
    "Los dinosaurios eran reptiles prehistóricos.",
    "La velocidad de un velociraptor era impresionante.",
    "Un triceratops tenía tres cuernos en la cabeza.",
    "El Tiranosaurio Rex fue uno de los depredadores más temidos.",
    "Los fósiles nos ayudan a entender la historia de los dinosaurios.",
]

resultados_nn = []
for frase in frases:
    pp_nn = calcular_perplejidad_nn(model, frase, vocab)
    resultados_nn.append({"Frase": frase, "Perplejidad PyTorch (Red Neuronal)": pp_nn})

# Convertir a DataFrame y mostrar
df_resultados_nn = pd.DataFrame(resultados_nn)
print(df_resultados_nn)


                                               Frase  \
0       Los dinosaurios eran reptiles prehistóricos.   
1  La velocidad de un velociraptor era impresiona...   
2    Un triceratops tenía tres cuernos en la cabeza.   
3  El Tiranosaurio Rex fue uno de los depredadore...   
4  Los fósiles nos ayudan a entender la historia ...   

   Perplejidad PyTorch (Red Neuronal)  
0                           44.644974  
1                          325.481877  
2                         6733.076966  
3                           91.511061  
4                         6870.055573  


In [91]:
def calcular_perplejidad_nn(model, frase, vocab, N=3):
    """Calcula la perplejidad de una frase usando la red neuronal."""
    model.eval()
    # Extrae tokens básicos (aquí usando solo \w y apóstrofe)
    tokens_frase = tokens_a_indices(re.findall(r"\b\w+(?:'\w+)?\b", frase.lower()), vocab)

    log_prob_sum = 0
    total_tokens = len(tokens_frase) - N

    if total_tokens <= 0:
        return float('inf')  # No hay suficiente contexto

    with torch.no_grad():
        for i in range(total_tokens):
            # Extraer contexto de tamaño N-1
            contexto = torch.tensor(tokens_frase[i:i+N-1]).unsqueeze(0).to(device)
            target = tokens_frase[i+N-1]

            output = model(contexto)
            prob = torch.softmax(output, dim=1)[0, target].item()

            log_prob_sum += math.log2(prob) if prob > 0 else -100  # Evita log(0)

    return 2 ** (-log_prob_sum / total_tokens)

###########################################
# 2) Cargar modelo KenLM
###########################################
kenlm_model = kenlm.LanguageModel('/content/drive/MyDrive/modelo_3_wiki.bin')

###########################################
# 3) Frases de prueba
###########################################
frases = [
    "Los dinosaurios eran reptiles prehistóricos.",
    "La velocidad de un velociraptor era impresionante.",
    "Un triceratops tenía tres cuernos en la cabeza.",
    "El Tiranosaurio Rex fue uno de los depredadores más temidos.",
    "Los fósiles nos ayudan a entender la historia de los dinosaurios.",
]

###########################################
# 4) Comparar perplejidades y armar DataFrame
###########################################
resultados_comparados = []

for frase in frases:
    # Perplejidad con la Red Neuronal (PyTorch)
    pp_nn = calcular_perplejidad_nn(model, frase, vocab, N=3)

    # Perplejidad con KenLM
    pp_kenlm = kenlm_model.perplexity(frase)

    resultados_comparados.append({
        "Frase": frase,
        "Perplejidad PyTorch (NN)": pp_nn,
        "Perplejidad KenLM": pp_kenlm
    })

df_resultados_comparados = pd.DataFrame(resultados_comparados)
print(df_resultados_comparados)

                                               Frase  \
0       Los dinosaurios eran reptiles prehistóricos.   
1  La velocidad de un velociraptor era impresiona...   
2    Un triceratops tenía tres cuernos en la cabeza.   
3  El Tiranosaurio Rex fue uno de los depredadore...   
4  Los fósiles nos ayudan a entender la historia ...   

   Perplejidad PyTorch (NN)  Perplejidad KenLM  
0                 44.644974        1102.775590  
1                325.481877         803.635915  
2               6733.076966        1544.857942  
3                 91.511061        1113.021586  
4               6870.055573         576.507724  


La disparidad en los resultados demuestra que cada modelo tiene regiones del espacio lingüístico en las que se desenvuelve mejor y otras en las que se queda corto. Ver perplejidades especialmente altas en ciertas frases sugiere que el modelo no ha visto suficiente evidencia (o la tokenización no le favorece), mientras que los valores bajos apuntan a un buen ajuste a esos contextos y vocabularios concretos.

PyTorch (NN) supera a KenLM en algunas frases, posiblemente debido a un mejor aprendizaje de contextos específicos o un corpus que cubría bien ciertos términos como “velociraptor”.
KenLM muestra una ventaja notable en otras frases (“triceratops”, “fósiles”), probablemente porque su corpus estadístico o smoothing abarca con mayor frecuencia esos temas.


**Conclusiones sobre el uso de redes neuronales (PyTorch) vs. KenLM para entrenar modelos de lenguaje**

1. **Enfoque y fundamentos distintos**  
   - **KenLM** se basa en modelos estadísticos de *n*-gramas con técnicas de *smoothing* (p. ej., Kneser-Ney). Estos modelos cuentan frecuencias de secuencias de palabras (o tokens) en un corpus y asignan probabilidades a las frases a partir de esos recuentos.  
   - **Redes neuronales (PyTorch)**, en cambio, representan el texto mediante *embeddings* y extraen patrones contextuales a través de arquitecturas como LSTM, GRU o Transformers. Este enfoque permite capturar mejor las dependencias a largo plazo y las relaciones semánticas entre palabras.

2. **Ventajas de KenLM**  
   - **Rapidez** y **simplicidad** en el entrenamiento y la inferencia. Entrenar un modelo n-gram con KenLM suele ser muy eficiente, incluso para corpus extensos.  
   - **Tamaño controlado de modelo**: Al ser *n*-gram, el almacenamiento se gestiona principalmente con estructuras comprimidas de conteos de secuencias.  
   - **Buen desempeño** en contextos de longitud **limitada** (generalmente hasta 5-gram o 7-gram), y si el corpus de entrenamiento es suficientemente grande y está representado en el dominio relevante.

3. **Ventajas de las redes neuronales (PyTorch)**  
   - **Capacidad de generalización**: Pueden captar relaciones a largo plazo y matices léxico-semánticos que no se capturan con conteos de frecuencias.  
   - **Flexibilidad** en la arquitectura: Desde un LSTM/GRU simple hasta modelos Transformer, lo que permite adaptar el enfoque a distintas tareas (modelado de lenguaje, clasificación, generación de texto, etc.).  
   - **Aprovechamiento de grandes corpus**: Mientras más datos, mejor suelen desempeñarse (cuando se entrena adecuadamente), a diferencia de los n-gramas que pueden saturarse o requerir grandes estructuras de memoria.

4. **Desventajas y consideraciones**  
   - **KenLM**:  
     - No es tan potente al capturar dependencias de oraciones muy largas o estructuras gramaticales complejas.  
     - Necesita un orden *n*-gram mayor para abarcar contextos más largos, pero esto incrementa rápidamente el tamaño del modelo y el riesgo de datos escasos (por lo que hay que usar *smoothing* avanzado).  
   - **Redes neuronales en PyTorch**:  
     - Requieren más **recursos computacionales** (CPU/GPU) y un conocimiento más profundo de hiperparámetros (tasa de aprendizaje, número de capas, tamaño de embeddings, etc.).  
     - El entrenamiento puede ser **lento** para corpus muy grandes si no se dispone de una infraestructura adecuada (GPUs).  
     - Si el corpus es muy pequeño o no está bien representado, la red puede sobreajustarse o no converger bien.

5. **Recomendaciones finales**  
   - **Usar KenLM** (o en general un modelo n-gram) cuando se necesite:  
     - Un sistema rápido y fácil de desplegar, con contextos de longitud modesta.  
     - Construir, por ejemplo, un módulo de corrección ortográfica o un sistema de autocompletado ligero para un dominio bien definido y con pocas variaciones.  
   - **Usar redes neuronales (PyTorch)** cuando se desee:  
     - Manejar vocabularios amplios, estructuras gramaticales complejas y dependencia a largo plazo.  
     - Realizar tareas de **generación de texto** más sofisticadas (p. ej., Transformers como GPT).  
     - Un sistema escalable a múltiples dominios, que aprenda matices semánticos y que pueda integrarse con otras tareas (traducción, resumen, clasificación de texto, etc.).

En definitiva, **la elección entre KenLM y una red neuronal en PyTorch** dependerá de **los recursos disponibles**, la **complejidad del lenguaje** que se quiera modelar y la **naturaleza de la tarea**. Para prototipos rápidos o contextos muy focalizados (y con poca capacidad de cómputo), KenLM es excelente. Para **capturar interacciones más profundas** en el texto y aprovechar un gran corpus con dependencia a largo plazo, el **enfoque neuronal** en PyTorch resulta más versátil y poderoso.